# Web Scraping : Hemnet.se

## Libraries: 

In [19]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
import requests

from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager # Setup Chrome 

options = webdriver.ChromeOptions()
options.add_argument('ignore-certificate-errors')

from selenium.webdriver.support.ui import Select #to press on the keyboard
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time

import re

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

import random



## Read in the csv containing the list of all Municipalities in Sweden

In [2]:
path = "Sweden_municipalities_data/Sweden_Municipalities_Main.csv"

# read the data as a list - easier to loop through it than a pandas frame
municipal = pd.read_csv(path,encoding="UTF-8").Municipality.tolist()
print(f"there are in total 290 municipalities in Sweden and we have : {len(municipal)}")

there are in total 290 municipalities in Sweden and we have : 290


# Start up the webdriver

In [605]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [606]:
# Maximize the window - otherwise we will get problems with the accessing the search bar
driver.maximize_window()

In [607]:
base_url = 'https://www.hemnet.se/salda/bostader'
driver.get(base_url)

### --- PopUp frame: click on the "Jag samtycker" button to get rid of it 

In [608]:
try:
    driver.find_element(
        By.XPATH,"//button[@ class = 'hcl-button hcl-button--primary']").click()
except Exception:
    pass 

# Enter in the first Municipality (Stokholms Kommun) into the search bar and search: Prototype - Ceck for the number of results

We will start with **Stockholm kommun** because its one of the largest and one of the few municipalities were we need to make the most effort in utilizing the filters on the website in order to get all the data wereas each search only shows 2500 results and therefore we must brake it down between type of real-estate, number of rooms, size in m^2 and slutprice

In [7]:
first_search = municipal[198]
first_search

'Stockholm Kommun'

In [610]:
# enter into the search bar

content = driver.find_element(By.CLASS_NAME,"qa-autocomplete-location")
content.clear()
content.send_keys("first_search")

#content.send_keys(Keys.ARROW_DOWN) # in the browser they already select the correct item - if we go down we select wrong item
time.sleep(3) # GIVE IT TIME TO LOAD THE RESULT

content.send_keys(Keys.ENTER)

Press enter

In [ ]:
# Press the search button "Hitta slutpriser"

# driver.find_element(
#     By.XPATH,"//button[@ class = 'hcl-button hcl-button--primary hcl-button--full-width search-form__submit-button qa-search-form-submit-button']"
# ).click()

# better classification: 
driver.find_element(
    By.XPATH,"//button[@type = 'submit']"
).click()

In [ ]:
# get the number of search results 

search_count = driver.find_element(
    By.XPATH,
    "//div[@ class ='result-type-toggle__count js-result-type-toggle-sold-hits qa-total-sold-hits']").text.replace(" ",'')
search_count = int(search_count)
search_count

For StockHolms Kommun we get 180.749 results of sold real-estate. The problem here is that the website only displays 2500 results were each page displays only 50 items and we get therefore only 50 pages. In order to go around this we can put more filters to our search e.g. to run a loop that goes splits the 180.749 into Villor, radhus, Lagenheter etc. and if the search result is still above 2500 we go through another loop splitting that data depending on the number of Rooms ("rum") there are in the real-estate. If that doesnt work we move to the size (Boarea) etc.  

In [9]:
#click on Villor button 
driver.find_element(By.XPATH,"//label[@for='search_item_types_villa']").click()

# click on submit search button 
driver.find_element(By.XPATH,"//button[@type = 'submit']").click()

# delay by 3 seconds - so that the website can properly load all the data
time.sleep(3)

# get the number of search results
search_count_villa = driver.find_element(
    By.XPATH,
    "//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ','')
search_count_villa

# OLD WAY
# search_count_villa = driver.find_element(
#     By.XPATH,
#     "//div[@ class ='result-type-toggle__count js-result-type-toggle-sold-hits qa-total-sold-hits']").text.replace(" ",'')
# search_count_villa = int(search_count_villa)
# print(search_count_villa)   

'6376'

in the categorie Villor  for bostadstyp we get 6.376 results which again is to much - add a filter for the number of rooms and see if thats enough:

I will increment the min and max by 1 step at the same time, e.g. 1-1, 1.5-1.5, 2-2, 2.5-2.5... etc up to 15 rooms

In [34]:
room_number = ["min","1","1.5","2","2.5","3","3.5","4","5","6","7","8"]

# increment except for room = 8,10 and 15 - instead for those run 2 times for 8-15
i = 0
while(i<11): # 15 because the maximum number is 15 - starts from zero
    if(room_number[i] == "8"): # set from 8 rooms to 10 rooms
        # set room min = 1
        select_min = Select(driver.find_element(By.XPATH,"//select[@name ='rooms_min']"))
        select_min.select_by_value("8")
        time.sleep(3)

        # set room max = 1
        select_max = Select(driver.find_element(By.XPATH,"//select[@name ='rooms_max']"))
        select_max.select_by_value("15")
        time.sleep(3)
        
        # search 
        driver.find_element(
        By.XPATH,"//button[@type = 'submit']").click()
        
        # delay
        time.sleep(3) #3 seconds

        # get the number of search results:
        search_count_villa_rum1  = driver.find_element(By.XPATH,
                                        "//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ','')
        search_count_villa_rum1 = int(search_count_villa_rum1)
        print(f"room min:8 max:15 => {search_count_villa_rum1}")
        
        i +=1


        # from Min to 1 - some real-estate have no rooms e.g. studio apartments or houses...?
    if(room_number[i] == "min"): # set from 8 rooms to 10 rooms
        # set room min = 1
        select_min = Select(driver.find_element(By.XPATH,"//select[@name ='rooms_min']"))
        select_min.select_by_visible_text("Min")
        time.sleep(3)

        # set room max = 1
        select_max = Select(driver.find_element(By.XPATH,"//select[@name ='rooms_max']"))
        select_max.select_by_value(room_number[i+1])
        time.sleep(3)
        
        # search 
        driver.find_element(
        By.XPATH,"//button[@type = 'submit']").click()
        
        # delay
        time.sleep(3) #3 seconds

        # get the number of search results:
        search_count_villa_rum1  = driver.find_element(By.XPATH,
                                        "//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ','')
        search_count_villa_rum1 = int(search_count_villa_rum1)
        print(f"room min:Min max:1 =>{search_count_villa_rum1}")
        
        i +=2
        
    if(room_number[i] == "7"): # set from 8 rooms to 10 rooms
        # set room min = 1
        select_min = Select(driver.find_element(By.XPATH,"//select[@name ='rooms_min']"))
        select_min.select_by_value(room_number[i])
        time.sleep(3)

        # set room max = 1
        select_max = Select(driver.find_element(By.XPATH,"//select[@name ='rooms_max']"))
        select_max.select_by_value(room_number[i])
        time.sleep(3)
        
        # search 
        driver.find_element(
        By.XPATH,"//button[@type = 'submit']").click()
        
        # delay
        time.sleep(3) #3 seconds

        # get the number of search results:
        search_count_villa_rum1  = driver.find_element(By.XPATH,
                                        "//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ','')
        search_count_villa_rum1 = int(search_count_villa_rum1)
        print(f"room min:7 max:7 =>{search_count_villa_rum1}")
        
        i +=1
        
#     elif(i == "10"): # set form 10 rooms - to 15 rooms
#             # set room min = 1
#         select_min = Select(driver.find_element(By.XPATH,"//select[@name ='rooms_min']"))
#         select_min.select_by_value("10")
#         time.sleep(3)

#         # set room max = 1
#         select_max = Select(driver.find_element(By.XPATH,"//select[@name ='rooms_max']"))
#         select_max.select_by_value("15")
#         time.sleep(3)
#         # search 
#         driver.find_element(
#             By.XPATH,"//button[@ class = 'hcl-button hcl-button--primary hcl-button--full-width search-form__submit-button qa-search-form-submit-button']").click()
#         # delay
#         time.sleep(3) #3 seconds

#         # get the number of search results:
#         search_count_villa_rum1 = driver.find_element(
#             By.XPATH,
#             "//div[@ class ='result-type-toggle__count js-result-type-toggle-sold-hits qa-total-sold-hits']").text.replace(" ",'')
#         search_count_villa_rum1 = int(search_count_villa_rum1)
#         print(search_count_villa_rum1) 

    else:
        # set room min = 1
        select_min = Select(driver.find_element(By.XPATH,"//select[@name ='rooms_min']"))
        select_min.select_by_value(room_number[i])
        time.sleep(3)

        # set room max = 1
        select_max = Select(driver.find_element(By.XPATH,"//select[@name ='rooms_max']"))
        select_max.select_by_value(room_number[i+1])
        time.sleep(3)
        
        # search 
        driver.find_element(
        By.XPATH,"//button[@type = 'submit']").click()
        
        # delay
        time.sleep(3) #3 seconds

        # get the number of search results:
        search_count_villa_rum1  = driver.find_element(By.XPATH,
                                        "//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ','')
        search_count_villa_rum1 = int(search_count_villa_rum1)
        print(f"room min:{room_number[i]} max:{room_number[i+1]} =>{search_count_villa_rum1}")
        
        i +=2
        print(i)


room min:Min max:1 =>101
room min:1.5 max:2 =>10
4
room min:2.5 max:3 =>158
6
room min:3.5 max:4 =>627
8
room min:5 max:6 =>3445
10
room min:7 max:7 =>1218


### From looks of things we can utilize the url link by adding the filters such as number of rooms etch
### to get the current url each time we can use driver.current_url

# PLAN A.1 : Building the Core function 

The core function simply takes the string of a municipality, searches that municipality and takes the number of search results and checks if its equal or less than 2500. that would be **step 1**. **Step 2** would be to brake the data down to Boastadstyp (villor, radhus, lagenheter, fritidsbenden, tomter, gardar/skogar and Övriga). If any of those categories has search results above 2500 we go to **stage 3** where we brake that category into number of rooms. If any of the room combination gives search result more than 2500 we move to **stage 4** and brake that into size (Boarea). If the size still gives search result greater than 2500 then we move to **stage 5** and we brake that down to Avgift. If Avgift gives search result greater than 2500 we move to **stage 6** we we brake the data into slutpris. 


**stage 0:** Base URL
    
            URL: https://www.hemnet.se/salda/bostader
    

**stage 1:** Municipality (290 instances): 
        
        Example: Stockholms Kommun (one of the biggest municipality - population 955.397 13th sept 2013) 

* *url argument:*    ?location_ids%5B%5D= "id NUMBER" (special id number only available after searching municipalitiy) 

* *URL example :*    https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031 


**stage 2:** Bostadstyp (7 instances):
    
        interface input: Villor, Radhus, Lagenheter,  Fritidsboenden, Tomter, Gardar/skogar, Övriga
              Url input: villa , radhus, bostadsratt, fritidshus    , tomt  , gard         , other

* *Url arguments:* &item_types%5B%5D= "categorie"  

* *Url exmple:* https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=other



**stage 3:** Rum (min-max) (9 instances)     :  

        Min-1, 1.5-1.5, 2-2, 2.5-2.5, 3-3, 3.5-3.5, 4-5, 6-7, 8-15, 16-max
        
* url arguments: &rooms_min="number"  and &rooms_max="number"

* url example: https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&rooms_min=1&rooms_max=15
        
**stage 4:** Boarea (min-max) (24 instances) - increment by 10m^2:  

        Min-30, 31-40, 41-50, 51-60, 61-70, 71-80, 81-90, 91-100, 101-110, 111-120, 121-130, 131-140, 141-150, 151-160,
        161-170, 171-180, 181-190, 191-200, 201-210, 211-220, 221-230, 231-240, 241-250, 251-max

*  url arguments: &living_area_min="number" and &living_area_max="number"

*  url example: https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&living_area_min=20&living_area_max=45
        
**stage 5:** Avgift (21 instances)            :

        Min-500, 501-1000, 1001-1500, 1501-2000, 2001-2500, 2501-3000, 3001-3500, 3501-4000, 4001-4500, 4501-5000,
        5001-5500, 5501-6000, 6001-6500, 6501-7000, 7001-7500, 7501-8000, 8001-8500, 8501-9000, 9001-9500, 
        9501-10000, 10001-20000, 20001-max

*  url arguments: &fee_min="number" and fee_max= "number"

*  url example: https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&fee_min=0&fee_max=1500

**stage 6:** Slutpris ( instances)           :
        
        Min-1000000, 1.000.001-2.000.000, 2000.001-3000.000, 3.000.001-4.000.000, 4.000.001-5.000.000, 
        5.000.001-6.000.000, 6.000.001-7.000.000, 7.000.001-8.000.000, 8.000.001-9.000.000, 
        9.000.001-10.000.000, 10.000.001-11.000.000, 11.000.001-12.000.000, 12.000.001-13.000.000,
        13.000.001-14.000.000, 14.000.001-15.000.000, 15.000.001-16.000.000, 16.000.001-max
        
*  url arguments: selling_price_min= "number"& and selling_price_max= "number"

*  url example: 
       https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&selling_price_min=300000&selling_price_max=4500000


## TEST - go through all manicipalities and count the serach result of the final stage that gives less or equal to 2500 results

### -- Setup the web driver

In [24]:
driver = webdriver.Chrome(ChromeDriverManager().install())

# Maximize the window - otherwise we will get problems with the accessing the search bar
driver.maximize_window()

base_url = 'https://www.hemnet.se/salda/bostader'
driver.get(base_url)

# sleep for 3 second then close the popup window
time.sleep(3)

try:
    driver.find_element(
        By.XPATH,"//button[@ class = 'hcl-button hcl-button--primary']").click()
except Exception:
    pass 

###########################################################################################

test:

In [10]:
# Search the given municipal (variable= area)
area = municipal[198] 
content = driver.find_element(By.CLASS_NAME,"qa-autocomplete-location")
content.send_keys(area)


In [13]:

content.send_keys(Keys.ENTER)


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=109.0.5414.75)


In [14]:

# press enter:

#         driver.find_element(
#             By.XPATH,"//button[@type = 'submit']"
#         ).click()
print("pressing search button")
driver.find_element(
    By.XPATH,"//button[@class='hcl-button hcl-button--primary hcl-button--full-width search-form__submit-button qa-search-form-submit-button']"
).click()



pressing search button


In [ ]:


# delay 3 seconds: 

# Stage 1: Count the number of research results:
print("counting result")
stage1_search_count_parent = driver.find_element(By.CLASS_NAME("qa-search-results-container js-sticky-scroll-trigger"))

stage1_search_count_child = int(stage1_search_count_parent.find_element(
                        By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

print(f'Stage 1 result count: {stage1_search_count}')

###########################################################################################

### -- get the list with all the manicipalities

In [25]:
path = "Sweden_municipalities_data/Sweden_Municipalities_Main.csv"

# read the data as a list - easier to loop through it than a pandas frame
municipal = pd.read_csv(path,encoding="UTF-8").Municipality.tolist()

print(f"there are in total 290 municipalities in Sweden and we have : {len(municipal)}")

there are in total 290 municipalities in Sweden and we have : 290


### -- Define the stage 2: real estate category as a lists

In [66]:
"""Stage 2: Bostadstyp"""
house_category = ['villa','radhus','bostadsratt','fritidshus','tomt','gard','other']
print(len(house_category))

'''Stage 3: number of rooms'''
rooms = [['min',1], [1.5,1.5], [2,2], [2.5,2.5], [3,3], [3.5,3.5], [4,5], [6,7], [8,15], [16,'max']]
print(len(rooms))

'''Stage 4: Boarea/ size'''
size = [['min',30], [31,40], [41,50], [51,60], [61,70], [71,80], [81,90], [91,100], [101,110], [111,120], [121,130], [131,140], [141,150], [151,160],[161,170], [171,180], [181,190], [191,200], [201,210], [211,220], [221,230], [231,240], [241,250], [251,'max']]
print(len(size))

'''Stage 5: Avgift'''
avgift = [["min",500], [501,1000], [1001,1500], [1501,2000], [2001,2500], [2501,3000], [3001,3500], [3501,4000], [4001,4500], [4501,5000],[5001,5500], [5501,6000],[6001,6500], [6501,7000], [7001,7500], [7501,8000], [8001,8500], [8501,9000], [9001,9500],[9501,10000], [10001,20000], [20001,'max']]
print(len(avgift))

'''Stage 6: finalprice'''
finalprice = [['min',1000000], [1000001,2000000], [2000001,3000000], [3000001,4000000], [4000001,5000000],[5000001,6000000], [6000001,7000000], [7000001,8000000], [8000001,9000000], [9000001,10000000], [10000001,11000000], [11000001,12000000], [12000001,13000000],[13000001,14000000], [14000001,15000000], [15000001,16000000], [16000001,'max']]
print(len(finalprice))

7
10
24
22
17


In [82]:
def count_results():
    listplace = []
    listStageNumber = []
    listStageName = []
    listCategoryMinmax = [] 
    listResult = []
    
    
    for municipal_number in range(1):
        
        area = municipal[198] # <-------------------------------testing for "stockholms kommun"
        print(f"Municipality: {area} ")

        # Search the given municipal (variable= area)

        content = driver.find_element(By.CLASS_NAME,"qa-autocomplete-location")
        content.send_keys(area)

        time.sleep(3) # GIVE IT TIME TO LOAD THE RESULT

        content.send_keys(Keys.ENTER)

        # press enter:
            # another way: simply press enter again because after the first enter the website takes you straight to the search button
        content.send_keys(Keys.ENTER)

        # Original solutions - but sometimes the don't fully complete and just keep running
        #         driver.find_element(
        #             By.XPATH,"//button[@type = 'submit']"
        #         ).click()
        print("pressing search button")
        #         driver.find_element(
        #             By.XPATH,"//button[@class='hcl-button hcl-button--primary hcl-button--full-width search-form__submit-button qa-search-form-submit-button']"
        #         ).click()

        
        # delay 3 seconds: 
        time.sleep(3)

        # Stage 1: Count the number of research results:
        print("counting result")
        stage1_search_count = int(driver.find_element(
                                By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

        print(f'Stage 1 result count: {stage1_search_count}')
        
        #add to list
        listplace.append(municipal[198])
        listStageNumber.append(1)
        listStageName.append('Municipalities')
        listCategoryMinmax.append('0')
        listResult.append(stage1_search_count)

        # if search result is smaller than 2500 then scrape - now just print
        print("if statement")
        if(stage1_search_count<=2500):
            print(f'Completet at stage 1. Search results in stage 1 = {stage1_search_count}')
        # Stage 2: If stage 1 result count is greater than 2500 move to stage 2: rooms
        else:
            print("stage 2")
          
            # Get the current url and set it as the base URL
            stage1_url = driver.current_url

            # Move to stage 2 - the categories
            for house_type in range(7):
                category = house_category[house_type]  

                # Set the URL to the category URL
                stage2_url = stage1_url + '&item_types%5B%5D='+category

                # Go to the website of the url:
                print()
                time.sleep(3) 
                driver.get(stage2_url)
                time.sleep(3)

                # Count the number of research results of stage 2:
                print("counting results")
                stage2_search_count = int(driver.find_element(
                                           By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

                print(f'Stage 2, category {category} - result count: {stage2_search_count}')
                
                #add to list
                listplace.append(municipal[198])
                listStageNumber.append(2)
                listStageName.append('House Type')
                listCategoryMinmax.append(category)
                listResult.append(stage2_search_count)

                # if search result is smaller than 2500 then scrape - now just print
                print("if statement")
                if(stage2_search_count<=2500):
                    print(f'Completed at stage 2. Search results in stage 2 for category {category} = {stage2_search_count}')

                # If not move to stage 3
                # Stage 3: If the given category in stage 2 has result count greater than 2500
                # move to stage 3: number of rooms
                else:
                    print("stage 3")
                    # get current URL to add on it.
                    stage2_url = driver.current_url

                    for minmax in range(10):
                        room_minmax = rooms[minmax]

                        # Set the URL to the category URL
                        stage3_url = stage2_url + '&rooms_min='+ str(room_minmax[0]) + '&rooms_max='+ str(room_minmax[1])

                        # Go to the website of the url: 
                        time.sleep(2)
                        driver.get(stage3_url)
                        time.sleep(5)

                        # Count the number of research results of stage 2:
                        print("counting results")
                        stage3_search_count = int(driver.find_element(
                                                    By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

                        print(f'Stage 3, Room minmax set {room_minmax} - result count: {stage3_search_count}')
                        
                        #add to list
                        listplace.append(municipal[198])
                        listStageNumber.append(3)
                        listStageName.append('Number of Rooms')
                        listCategoryMinmax.append(str('MIN: '+room_minmax[0])+'-'+'MAX: '+str(room_minmax[1]))
                        listResult.append(stage3_search_count)

                        print("if statement")
                        if(stage3_search_count<=2500):
                            print(f'Completed at stage 3. Search results in stage 3 for room minmax set {room_minmax} = {stage3_search_count}')

                        # Stage 4: If the given room minmax set in stage 3 has result count greater than 2500 
                        # move to stage 4 - size
                        else:
                            print("stage 4")

                            # get current URL to add on it.
                            stage3_url = driver.current_url

                            for minmax in range(24):
                                size_minmax = size[minmax]

                                # Set the URL to the category URL
                                stage4_url = stage3_url + '&living_area_min=' + str(size_minmax[0]) + '&living_area_max='+ str(size_minmax[1])

                                # Go to the website of the url:
                                time.sleep(2) 
                                driver.get(stage4_url)
                                time.sleep(5)

                                # Count the number of research results of stage 2:
                                print("counting results")
                                stage4_search_count = int(driver.find_element(
                                                            By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

                                print(f'Stage 4, Size minmax set {size_minmax} - result count: {stage4_search_count}')
                                
                                #add to list
                                listplace.append(municipal[198])
                                listStageNumber.append(4)
                                listStageName.append("")
                                listCategoryMinmax.append(str('MIN: '+size_minmax[0])+'-'+'MAX:'+str(size_minmax[1]))
                                listResult.append(stage4_search_count)

                                print("if statement")
                                if(stage4_search_count<=2500):
                                    print(f'Completed at stage 4. Search results in stage 4 for size minmax set {size_minmax} = {stage4_search_count}')

                                # Stage 5: If the given size minmax set in stage 4 has result count greater than 
                                # 2500 move to stage 5 -Avgift
                                else:
                                    print("stage 5")
                                    # get current URL to add on it.
                                    stage4_url = driver.current_url

                                    for minmax in range(22):
                                        avgift_minmax = avgift[minmax]

                                        # Set the URL to the category URL
                                        stage5_url = stage4_url + '&fee_min=' + str(avgift_minmax[0]) + '&fee_max='+ str(avgift_minmax[1])

                                        # Go to the website of the url: 
                                        time.sleep(2)
                                        driver.get(stage5_url)
                                        time.sleep(5)

                                        # Count the number of research results of stage 2:
                                        print("counting results")
                                        stage5_search_count = int(driver.find_element(
                                                                    By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

                                        print(f'Stage 5, Avgift minmax set {avgift_minmax} - result count: {stage5_search_count}')
                                        
                                        #add to list
                                        listplace.append(municipal[198])
                                        listStageNumber.append(5)
                                        listStageName.append('Avgift')
                                        listCategoryMinmax.append(str('MIN: '+avgift_minmax[0])+'-'+'MAX: '+str(avgift_minmax[1]))
                                        listResult.append(stage5_search_count)

                                        print("if statement")
                                        if(stage5_search_count<=2500):
                                            print(f'Completed at stage 4. Search results in stage 4 for Avgift minmax set {avgift_minmax} = {stage5_search_count}')

                                        # Stage 6: If the given Avgift minmax set in stage 5 has result count greater than 
                                        # 2500 move to stage 5 - Slutpris / Finalprice
                                        else:
                                            print("stage 6")
                                            # get current URL to add on it.
                                            stage5_url = driver.current_url

                                            for minmax in range(17):
                                                finalprice_minmax = finalprice[minmax]

                                                # Set the URL to the category URL
                                                stage6_url = stage5_url + '&selling_price_min=' + str(finalprice_minmax[0]) + '&selling_price_max='+ str(finalprice_minmax[1])

                                                # Go to the website of the url: 
                                                time.sleep(2)
                                                driver.get(stage6_url)
                                                time.sleep(5)

                                                # Count the number of research results of stage 2:
                                                print("counting results")
                                                stage6_search_count = int(driver.find_element(
                                                                            By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

                                                print(f'Stage 6, Finalprice minmax set {finalprice_minmax} - result count: {stage6_search_count}')
                                                
                                                #add to list
                                                listplace.append(municipal[198])
                                                listStageNumber.append(6)
                                                listStageName.apppend('Finalprice')
                                                listCategoryMinmax.append(str('MIN: '+finalprice_minmax[0])+'-'+'MAX: '+str(finalprice_minmax[1]))
                                                listResult.append(stage6_search_count)
    # Export the result as csv file
    tableToCSV = pd.DataFrame({'Place': listplace, 'Stage_number': listStageNumber,
                               'Stage_name': listStageName, 'Category/MinMax': listCategoryMinmax,
                               'Number_of_Results': listResult})
    # write csv
    tableToCSV.to_csv('countData_Stockholms_Kommun.csv',sep='\t', encoding='utf-8')
                                
    return tableToCSV                       

In [73]:
testRun = count_results()

Municipality: Stockholm Kommun 
pressing search button
counting result
Stage 1 result count: 180780
if statement
stage 2

counting results
Stage 2, category villa - result count: 6377
if statement
stage 3
counting results
Stage 3, Room minmax set ['min', 1] - result count: 101
if statement
Completed at stage 3. Search results in stage 3 for room minmax set ['min', 1] = 101
counting results
Stage 3, Room minmax set [1.5, 1.5] - result count: 0
if statement
Completed at stage 3. Search results in stage 3 for room minmax set [1.5, 1.5] = 0
counting results
Stage 3, Room minmax set [2, 2] - result count: 10
if statement
Completed at stage 3. Search results in stage 3 for room minmax set [2, 2] = 10
counting results
Stage 3, Room minmax set [2.5, 2.5] - result count: 2
if statement
Completed at stage 3. Search results in stage 3 for room minmax set [2.5, 2.5] = 2
counting results
Stage 3, Room minmax set [3, 3] - result count: 156
if statement
Completed at stage 3. Search results in stage 3

counting results
Stage 4, Size minmax set [101, 110] - result count: 833
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [101, 110] = 833
counting results
Stage 4, Size minmax set [111, 120] - result count: 1007
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [111, 120] = 1007
counting results
Stage 4, Size minmax set [121, 130] - result count: 608
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [121, 130] = 608
counting results
Stage 4, Size minmax set [131, 140] - result count: 548
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [131, 140] = 548
counting results
Stage 4, Size minmax set [141, 150] - result count: 88
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [141, 150] = 88
counting results
Stage 4, Size minmax set [151, 160] - result count: 40
if statement
Completed at stage 4. Search results in stage 4 for size min

counting results
Stage 5, Avgift minmax set [5501, 6000] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [5501, 6000] = 0
counting results
Stage 5, Avgift minmax set [6001, 6500] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [6001, 6500] = 0
counting results
Stage 5, Avgift minmax set [6501, 7000] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [6501, 7000] = 0
counting results
Stage 5, Avgift minmax set [7001, 7500] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [7001, 7500] = 0
counting results
Stage 5, Avgift minmax set [7501, 8000] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [7501, 8000] = 0
counting results
Stage 5, Avgift minmax set [8001, 8500] - result count: 0
if statement
Completed at stage 4. Search results 

counting results
Stage 5, Avgift minmax set [10001, 20000] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [10001, 20000] = 0
counting results
Stage 5, Avgift minmax set [20001, 'max'] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [20001, 'max'] = 0
counting results
Stage 4, Size minmax set [41, 50] - result count: 2071
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [41, 50] = 2071
counting results
Stage 4, Size minmax set [51, 60] - result count: 115
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [51, 60] = 115
counting results
Stage 4, Size minmax set [61, 70] - result count: 9
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [61, 70] = 9
counting results
Stage 4, Size minmax set [71, 80] - result count: 11
if statement
Completed at stage 4. Search results in stage 4 for size min

counting results
Stage 4, Size minmax set [41, 50] - result count: 2184
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [41, 50] = 2184
counting results
Stage 4, Size minmax set [51, 60] - result count: 190
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [51, 60] = 190
counting results
Stage 4, Size minmax set [61, 70] - result count: 5
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [61, 70] = 5
counting results
Stage 4, Size minmax set [71, 80] - result count: 1
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [71, 80] = 1
counting results
Stage 4, Size minmax set [81, 90] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [81, 90] = 0
counting results
Stage 4, Size minmax set [91, 100] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for size minmax set [91, 100] = 0
counting r

...

counting results
Stage 5, Avgift minmax set [7001, 7500] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [7001, 7500] = 0
counting results
Stage 5, Avgift minmax set [7501, 8000] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [7501, 8000] = 0
counting results
Stage 5, Avgift minmax set [8001, 8500] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [8001, 8500] = 0
counting results
Stage 5, Avgift minmax set [8501, 9000] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [8501, 9000] = 0
counting results
Stage 5, Avgift minmax set [9001, 9500] - result count: 0
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [9001, 9500] = 0
counting results
Stage 5, Avgift minmax set [9501, 10000] - result count: 0
if statement
Completed at stage 4. Search results

counting results
Stage 6, Finalprice minmax set ['min', 1000000] - result count: 0
counting results
Stage 6, Finalprice minmax set [1000001, 2000000] - result count: 178
counting results
Stage 6, Finalprice minmax set [2000001, 3000000] - result count: 818
counting results
Stage 6, Finalprice minmax set [3000001, 4000000] - result count: 700
counting results
Stage 6, Finalprice minmax set [4000001, 5000000] - result count: 531
counting results
Stage 6, Finalprice minmax set [5000001, 6000000] - result count: 469
counting results
Stage 6, Finalprice minmax set [6000001, 7000000] - result count: 370
counting results
Stage 6, Finalprice minmax set [7000001, 8000000] - result count: 210
counting results
Stage 6, Finalprice minmax set [8000001, 9000000] - result count: 101
counting results
Stage 6, Finalprice minmax set [9000001, 10000000] - result count: 41
counting results
Stage 6, Finalprice minmax set [10000001, 11000000] - result count: 18
counting results
Stage 6, Finalprice minmax se

counting results
Stage 5, Avgift minmax set [4001, 4500] - result count: 1895
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [4001, 4500] = 1895
counting results
Stage 5, Avgift minmax set [4501, 5000] - result count: 2141
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [4501, 5000] = 2141
counting results
Stage 5, Avgift minmax set [5001, 5500] - result count: 965
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [5001, 5500] = 965
counting results
Stage 5, Avgift minmax set [5501, 6000] - result count: 328
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [5501, 6000] = 328
counting results
Stage 5, Avgift minmax set [6001, 6500] - result count: 69
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [6001, 6500] = 69
counting results
Stage 5, Avgift minmax set [6501, 7000] - result count: 38
if statement
Completed at s

counting results
Stage 5, Avgift minmax set [4501, 5000] - result count: 1143
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [4501, 5000] = 1143
counting results
Stage 5, Avgift minmax set [5001, 5500] - result count: 767
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [5001, 5500] = 767
counting results
Stage 5, Avgift minmax set [5501, 6000] - result count: 321
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [5501, 6000] = 321
counting results
Stage 5, Avgift minmax set [6001, 6500] - result count: 114
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [6001, 6500] = 114
counting results
Stage 5, Avgift minmax set [6501, 7000] - result count: 26
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [6501, 7000] = 26
counting results
Stage 5, Avgift minmax set [7001, 7500] - result count: 11
if statement
Completed at sta

counting results
Stage 5, Avgift minmax set [5501, 6000] - result count: 978
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [5501, 6000] = 978
counting results
Stage 5, Avgift minmax set [6001, 6500] - result count: 317
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [6001, 6500] = 317
counting results
Stage 5, Avgift minmax set [6501, 7000] - result count: 196
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [6501, 7000] = 196
counting results
Stage 5, Avgift minmax set [7001, 7500] - result count: 70
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [7001, 7500] = 70
counting results
Stage 5, Avgift minmax set [7501, 8000] - result count: 21
if statement
Completed at stage 4. Search results in stage 4 for Avgift minmax set [7501, 8000] = 21
counting results
Stage 5, Avgift minmax set [8001, 8500] - result count: 12
if statement
Completed at stage 4

time_start 02:14


In [80]:
table = pd.read_csv('countData_Stockholms_Kommun.csv',sep="\t")
table

,Unnamed: 0,Place,Stage_number,Stage_name,Number_of_Results
0,0,Stockholm Kommun,1,Municipalities,180780
1,1,Stockholm Kommun,2,villa,6377
2,2,Stockholm Kommun,3,min-1,101
3,3,Stockholm Kommun,3,1.5-1.5,0
4,4,Stockholm Kommun,3,2-2,10
...,...,...,...,...,...
805,805,Stockholm Kommun,3,16-max,1
806,806,Stockholm Kommun,2,fritidshus,18
807,807,Stockholm Kommun,2,tomt,110
808,808,Stockholm Kommun,2,gard,0


## Creating and testing Scrape function - 3 tests

In [86]:
# Only 1 page - 47 items
url_1page = 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=other'
   
# 3 pages - 110 items
url_3pages = 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=tomt'

# 16 pages - 785 items
url_16pages = 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=villa&rooms_min=2.5&rooms_max=4'

###  -------------------------------------     test 1 : Only 1 page 47 items

In [88]:
response = requests.get(url_1page)
response

<Response [403]>

we get the response 403 - forbidden 
to go passed this we need to identify the User-agent on our header 

to find the header we go on any browser and enter: httpbin.org/get
There we need to copy the User-Agent and then insert it in the code.

In [90]:
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-63c40070-5d3770130fb764901d13ba81"}

response = requests.get(url_1page, headers=header)
response

<Response [200]>

In [142]:
# create beautifulsoup object from the data
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<!--[if IE 8 ]>    <html lang="sv-SE" dir="ltr" class="ie8"> <![endif]-->
<!--[if IE 9 ]>    <html lang="sv-SE" dir="ltr" class="ie9"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html dir="ltr" lang="sv-SE">
 <!--<![endif]-->
 <!--
           hhhhhhhh
      hhhhhhhhhhhhhhhhhh
    hhhhh             hhhh
  hhhhh                 hhhh
 hhhh         hhh        hhhh
 hhh       hhhhhhhhh      hhhh
hhhh     hhhhhhhhhhhh     hhhh
hhhh     hhhh    hhhh     hhhh
hhhh     hhhh    hhhh     hhhh
 hhhh                    hhhh
  hhhhh                 hhhh
   hhhhhh            hhhhh
 hhhhh hhhhhhhhhhhhhhhh
hhhh     hhhhhhhhhhh

Kul att du hittat hit! Vill du också jobba på en av Sveriges största sajter med miljontals unika besökare?
Gör en spontanansökan på https://career.hemnet.se/
-->
 <head>
  <meta charset="utf-8"/>
  <link href="//assets.hemnet.se" rel="dns-prefetch"/>
  <link href="//bilder.hemnet.se" rel="dns-prefetch"/>
  <link href="//maps.googleapis.com" rel="dns-prefetch"/>


In [247]:
# get all houses on the page 
results1 = soup.find_all('li', {'class': "sold-results__normal-hit"})
len(results)

47

Ok THIS LOOKS GOOD - WE GOT 47 ITEMS WHICH IS CORRESPONSE TO THE NUMBER OF RESULTS WE GOT

In [220]:
onehouse = results[1] # start by only one house

In [147]:
# get the link: 
link = onehouse.find('a', {'class': "sold-property-link js-sold-property-card-link"}).get('href')
link

'https://www.hemnet.se/salda/ovrigt-1rum-spanga-bromsten-stockholms-kommun-stridsbergavagen-2-4-9189425484463838645'

In [205]:
# get the house address
houseAddress_parent = onehouse.find('div', {'class': 'sold-property-listing__location'})

# first address
houseAddress_1 = houseAddress_parent.find('h2').text.strip()
print(houseAddress_1)

Stridsbergavägen 2-4


In [203]:
# SEcond Address
houseAddress_2 = houseAddress_parent.find('div').text.strip().replace("\n","")[13:].strip()
import re
houseAddress_2 = " ".join(re.split("\s+", houseAddress_2, flags=re.UNICODE)) # get rid of double white spaces
houseAddress_2

'Spånga - Bromsten, Stockholms kommun'

In [216]:
# get the size
size = onehouse.find('div',{'class':"sold-property-listing__subheading sold-property-listing__area"}).text.split()
size = 
#print(size.split())

['14', 'm²', '1', 'rum']

In [238]:
# better to take this data in the hyperlink

# # Get the Slutprice

# finalPrice = onehouse.find('div', {'class':"sold-property-listing__price"}).text.replace("\xa0000;","").strip()
# import unidecode

# # The most robust way would be to use the unidecode module to convert all 
# # non-ASCII characters to their closest ASCII equivalent automatically
# #  ----> remove this \xa0000 stuff 

# finalPrice = unidecode.unidecode(finalPrice)
# finalPrice = " ".join(re.split("\s+", finalPrice, flags=re.UNICODE)) 
# finalPrice

'Slutpris 90 000 kr Sald 31 oktober 2022'

In [240]:
test = "Övrig/okänd, bostadsrätt - Midsommarkransen/Telefonplan, Stockholms kommun - Såld den 19 augusti 2019"
test.split('-')[2]

' Såld den 19 augusti 2019'

In [231]:
import unidecode
word = unidecode.unidecode(finalPrice)
word

'Slutpris 90 000 kr\n        \n\n            Sald 31 oktober 2022'

In [249]:
# loop through all items and keep track
records = []
for item in results1:
    # get the link
    link = item.find('a', {'class': "sold-property-link js-sold-property-card-link"}).get('href')
   
    # get the house address
    houseAddress_parent = item.find('div', {'class': 'sold-property-listing__location'})
    
    # get the house address 1
    houseAddress_1 = houseAddress_parent.find('h2').text.strip()
    
    # get the house address 2
    houseAddress_2 = houseAddress_parent.find('div').text.strip().replace("\n","")[13:].strip()
    import re
    houseAddress_2 = " ".join(re.split("\s+", houseAddress_2, flags=re.UNICODE)) # get rid of double white spaces

    # create a tuple with the 2 data 
    data = (houseAddress_1,houseAddress_2, link)
    records.append(data)

In [242]:
len(records)

47

In [250]:
# " convert the list to panda dataframe"
testTable = pd.DataFrame(records, columns= ['House_Address_1','House_address_2', 'link'])
testTable

,House_Address_1,House_address_2,link
0,Krukmakarbacken 2 (Garageplats N08),"Södermalm, Stockholms kommun",https://www.hemnet.se/salda/ovrigt-1rum-soderm...
1,Stridsbergavägen 2-4,"Spånga - Bromsten, Stockholms kommun",https://www.hemnet.se/salda/ovrigt-1rum-spanga...
2,Kebnekaisevägen 7,"Traneberg, Stockholms kommun",https://www.hemnet.se/salda/ovrigt-2rum-traneb...
3,Lövåsvägen 45,"Ulvsunda, Stockholms kommun",https://www.hemnet.se/salda/ovrigt-350m2-ulvsu...
4,"Brantingsgatan 17, souterrängplan","Gärdet, Stockholms kommun",https://www.hemnet.se/salda/ovrigt-3rum-gardet...
5,David Bagares gata 26A,"City/Norrmalm/Vasastan, Stockholms kommun",https://www.hemnet.se/salda/ovrigt-3rum-city-n...
6,Sundbyvägen 70,"Spånga - Sundby, Stockholms kommun",https://www.hemnet.se/salda/ovrigt-2rum-spanga...
7,"Cedergrensvägen 61, Bostadsrättslokal","Telefonplan /Midsommarkransen, Stockholms kommun",https://www.hemnet.se/salda/ovrigt-0m2-telefon...
8,Alingsåsvägen 6,"Hammarbyhöjden, Stockholms kommun",https://www.hemnet.se/salda/ovrigt-1rum-hammar...
9,Ulvsundavägen 36,"Ulvsunda, Stockholms kommun",https://www.hemnet.se/salda/ovrigt-2rum-ulvsun...


###  -------------------------------------     test 2:  3 pages - 110 items

In [246]:
url_3pages = 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=tomt'

In [251]:
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-63c40070-5d3770130fb764901d13ba81"}

response = requests.get(url_3pages, headers=header)
response

<Response [200]>

In [253]:
# create beautifulsoup object from the data
soup = BeautifulSoup(response.content, 'html.parser')
#print(soup.prettify())

In [255]:
# get all houses on the page 
results2 = soup.find_all('li', {'class': "sold-results__normal-hit"})
len(results2)

50

Now we need to add a function that loops through all items on each page and stops when there are no more pages

In [269]:
# loop through all items and keep track
records_test2 = []
url = 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=tomt'

while True:
    """ Set up the page"""
    
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-63c40070-5d3770130fb764901d13ba81"}

    response = requests.get(url, headers=header)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    results2 = soup.find_all('li', {'class': "sold-results__normal-hit"})
    
    
    
    for item in results2:
        # get the link
        link = item.find('a', {'class': "sold-property-link js-sold-property-card-link"}).get('href')

        # get the house address
        houseAddress_parent = item.find('div', {'class': 'sold-property-listing__location'})

        # get the house address 1
        houseAddress_1 = houseAddress_parent.find('h2').text.strip()

        # get the house address 2
        houseAddress_2 = houseAddress_parent.find('div').text.strip().replace("\n","")[13:].strip()
        import re
        houseAddress_2 = " ".join(re.split("\s+", houseAddress_2, flags=re.UNICODE)) # get rid of double white spaces

        # create a tuple with the 2 data 
        data = (houseAddress_1,houseAddress_2, link)
        records_test2.append(data)
        
    try:    
        find_nextpage_url = soup.find('a', {'class' : 'next_page hcl-button hcl-button--primary hcl-button--full-width'})
        if find_nextpage_url.get('href'): # if there is a link
            url = 'https://www.hemnet.se/'+ find_nextpage_url.get('href')
            print(url)
        else:
            break
    except AttributeError: #if there is no next button link
        print("Done")
        break
    
    
test2_table = pd.DataFrame(records_test2, columns= ['House_Address_1','House_address_2', 'link'])



https://www.hemnet.se//salda/bostader?item_types%5B%5D=tomt&location_ids%5B%5D=18031&page=2
https://www.hemnet.se//salda/bostader?item_types%5B%5D=tomt&location_ids%5B%5D=18031&page=3
Done


In [266]:
len(records)

110

In [562]:
test2_table

,House_Address_1,House_address_2,link
0,Näckrosbrinken 18,"Hässelby Villastad, Stockholms kommun",https://www.hemnet.se/salda/tomt-hasselby-vill...
1,Ivarskärrsvägen 109 B (adress ej fastslaget),"Vällingby - Vinsta, Stockholms kommun",https://www.hemnet.se/salda/tomt-vallingby-vin...
2,Sörgårdsvägen 65A,"Spånga Solhem, Stockholms kommun",https://www.hemnet.se/salda/tomt-spanga-solhem...
3,Solrosvägen 86,"Älvsjö - Herrängen, Stockholms kommun",https://www.hemnet.se/salda/tomt-alvsjo-herran...
4,Solrosvägen 86,"Älvsjö - Herrängen, Stockholms kommun",https://www.hemnet.se/salda/tomt-alvsjo-herran...
...,...,...,...
890,Victor Balcks Väg 187,"Tallkrogen, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-tallkro...
891,Fållnäsgatan 50,"Svedmyra, Stockholms kommun",https://www.hemnet.se/salda/villa-3rum-svedmyr...
892,Annebodavägen 13,"Liseberg / Älvsjö, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-liseber...
893,Långängsvägen 76,"Älvsjö Herrängen, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-alvsjo-...


In [568]:
test2_table.iloc[50:100]

,House_Address_1,House_address_2,link
50,Långjökelgatan 2,"Kista, Stockholms kommun",https://www.hemnet.se/salda/tomt-kista-stockho...
51,Långjökelgatan 2,"Kista, Stockholms kommun",https://www.hemnet.se/salda/tomt-kista-stockho...
52,Långjökelgatan 2,"Kista, Stockholms kommun",https://www.hemnet.se/salda/tomt-kista-stockho...
53,Långjökelgatan 2,"Kista, Stockholms kommun",https://www.hemnet.se/salda/tomt-kista-stockho...
54,Långjökelgatan 2,"Kista, Stockholms kommun",https://www.hemnet.se/salda/tomt-kista-stockho...
55,Långjökelgatan 2,"Kista, Stockholms kommun",https://www.hemnet.se/salda/tomt-kista-stockho...
56,Långjökelgatan 2,"Kista, Stockholms kommun",https://www.hemnet.se/salda/tomt-kista-stockho...
57,Långjökelgatan 2,"Kista, Stockholms kommun",https://www.hemnet.se/salda/tomt-kista-stockho...
58,Långjökelgatan 2,"Kista, Stockholms kommun",https://www.hemnet.se/salda/tomt-kista-stockho...
59,Långjökelgatan 2,"Kista, Stockholms kommun",https://www.hemnet.se/salda/tomt-kista-stockho...


### HERE ABOVE : WEIRD!!!! BUT EACH PROPERTIE SEEMS TO BE DIFFERENT ... WHICH IT SHOULD BE

In [261]:
test = soup.find('a', {'class' : 'next_page hcl-button hcl-button--primary hcl-button--full-width'}).get('href')
test

'/salda/bostader?item_types%5B%5D=tomt&location_ids%5B%5D=18031&page=2'

###  -------------------------------------     test 3:  16 pages - 785 items

In [ ]:
# 16 pages - 785 items
url_16pages = 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=villa&rooms_min=2.5&rooms_max=4'

In [274]:
import re

# loop through all items and keep track
records_test3 = []
url = 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=villa&rooms_min=2.5&rooms_max=4'

while True:
    """ Set up the page"""
    
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-63c40070-5d3770130fb764901d13ba81"}

    response = requests.get(url, headers=header)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    results3 = soup.find_all('li', {'class': "sold-results__normal-hit"})
    
    
    
    for item in results3:
        # get the link of each item
        link = item.find('a', {'class': "sold-property-link js-sold-property-card-link"}).get('href')

        # get the house address of each item
        houseAddress_parent = item.find('div', {'class': 'sold-property-listing__location'})

        # get the house address 1
        houseAddress_1 = houseAddress_parent.find('h2').text.strip()

        # get the house address 2
        houseAddress_2 = houseAddress_parent.find('div').text.strip().replace("\n","")[13:].strip()
            # import re
        houseAddress_2 = " ".join(re.split("\s+", houseAddress_2, flags=re.UNICODE)) # get rid of double white spaces

        # create a tuple with the 2 data 
        data = (houseAddress_1,houseAddress_2, link)
        records_test3.append(data)
        
    try:    
        find_nextpage_url = soup.find('a', {'class' : 'next_page hcl-button hcl-button--primary hcl-button--full-width'})
        if find_nextpage_url.get('href'): # if there is a link
            url = 'https://www.hemnet.se/'+ find_nextpage_url.get('href')
            print(url)
        else:
            break
    except AttributeError: #if there is no next button link
        print("Done")
        break
    
    
test3_table = pd.DataFrame(records_test3, columns= ['House_Address_1','House_address_2', 'link'])

https://www.hemnet.se//salda/bostader?item_types%5B%5D=villa&location_ids%5B%5D=18031&page=2&rooms_max=4&rooms_min=2.5
https://www.hemnet.se//salda/bostader?item_types%5B%5D=villa&location_ids%5B%5D=18031&page=3&rooms_max=4&rooms_min=2.5
https://www.hemnet.se//salda/bostader?item_types%5B%5D=villa&location_ids%5B%5D=18031&page=4&rooms_max=4&rooms_min=2.5
https://www.hemnet.se//salda/bostader?item_types%5B%5D=villa&location_ids%5B%5D=18031&page=5&rooms_max=4&rooms_min=2.5
https://www.hemnet.se//salda/bostader?item_types%5B%5D=villa&location_ids%5B%5D=18031&page=6&rooms_max=4&rooms_min=2.5
https://www.hemnet.se//salda/bostader?item_types%5B%5D=villa&location_ids%5B%5D=18031&page=7&rooms_max=4&rooms_min=2.5
https://www.hemnet.se//salda/bostader?item_types%5B%5D=villa&location_ids%5B%5D=18031&page=8&rooms_max=4&rooms_min=2.5
https://www.hemnet.se//salda/bostader?item_types%5B%5D=villa&location_ids%5B%5D=18031&page=9&rooms_max=4&rooms_min=2.5
https://www.hemnet.se//salda/bostader?item_types

In [275]:
test3_table

,House_Address_1,House_address_2,link
0,Estlandsgatan 128,"Enskedefältet - Enskede, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-enskede...
1,Svenskbyvägen 25,"Sundby, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-sundby-...
2,Blackensvägen 54,"Älvsjö, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-alvsjo-...
3,Krokanvägen 18,"Sköndal, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-skondal...
4,Småbrukarvägen 13,"Sundby, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-sundby-...
...,...,...,...
780,Victor Balcks Väg 187,"Tallkrogen, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-tallkro...
781,Fållnäsgatan 50,"Svedmyra, Stockholms kommun",https://www.hemnet.se/salda/villa-3rum-svedmyr...
782,Annebodavägen 13,"Liseberg / Älvsjö, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-liseber...
783,Långängsvägen 76,"Älvsjö Herrängen, Stockholms kommun",https://www.hemnet.se/salda/villa-4rum-alvsjo-...


# Test - go into each property and collect more data:  

we will use the test link number 2 with 3 pages and 110 properties

link: 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=tomt'

In [576]:

# loop through all items and keep track
records_test4 = []
url = 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=tomt'

while True:
    """ Set up the page"""
    
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-63c40070-5d3770130fb764901d13ba81"}

    response = requests.get(url, headers=header)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    results4 = soup.find_all('li', {'class': "sold-results__normal-hit"})
    
    property_count = 0
    # *********************************************
    
    for item in results4:
        property_count += 1
        print(f"property number: {property_count} out of 110")
        
        # get the link of each item
        link = item.find('a', {'class': "sold-property-link js-sold-property-card-link"}).get('href')
        
        # Create property ID column - from the ID number in the end of each property URL
        propertyID_hemnet = link.split("-")[-1].strip()

        # get the house address of each item
        houseAddress_parent = item.find('div', {'class': 'sold-property-listing__location'})

        # get the house address 1
        houseAddress_1 = houseAddress_parent.find('h2').text.strip()
        print(houseAddress_1)
        # get the house address 2
        houseAddress_2 = houseAddress_parent.find('div').text.strip().replace("\n","")[13:].strip()
            # import re
        houseAddress_2 = " ".join(re.split("\s+", houseAddress_2, flags=re.UNICODE)) # get rid of double white spaces
        
        # ____________________________________________
        # Go into the property link and get more data
        #_____________________________________________
        item_url      = link
        time.sleep(5)
        item_response = requests.get(item_url, headers=header)
        item_soup     = BeautifulSoup(item_response.content, 'html.parser')
        
        
        # get the date the porperty was last sold
        item_date_sold_at = item_soup.find('time').text.strip()
        #****************************************
        
        # get the final price
        item_finalprice_raw = item_soup.find('span',{'class':'sold-property__price-value'}).text.strip()
        item_finalprice = " ".join(re.split("\s+", item_finalprice_raw, flags=re.UNICODE)).replace(" kr",'').replace(" ","")
        #****************************************
        
        # get all the other data with one call : 
            #  property type (Bostadstyp)   , Release form (Upplåtelseform), Number of rooms (Antal rum),
            #  size (Boarea)                , Biarea (Biarea)              ,  land area (Tomtarea), 
            #  Year of construction (Byggar), Fee/month (Avgift/månad), fee/year (Tomträttsavgäld)
        # All at on fetch - the key difference between properties is if there is "Biarea" and/or "Tomtarea"
        # then the list is longer. And the last category is different e.g. Avgift/månad or Tomträttsavgäld etc. in per mount
        # or in per year fees. 
                # collect the data
        item_data = item_soup.find('dl',{'class':'sold-property__attributes'})#.text.strip()

                # set up the data to run the forloop of the categories
        # Category names:
        item_data_dt = item_data.find_all('dt')
        item_data_Categorynames = []
        for category in item_data_dt:
            category = category.text
            item_data_Categorynames.append(category)
        item_data_Categorynames
        
        # Category Values:
        item_data_dd = item_data.find_all('dd') # values
        item_data_CategoryValues = []
        for value in item_data_dd:
            value = value.text.strip()
            item_data_CategoryValues.append(value)
        item_data_CategoryValues
        
        # Initiate a zero value (0) to all the categories before setting the actual values in.
            # Because if a property does not e.g. have "byggar" then at least it will be 0 instead of plank
        item_Bostadstyp      = 0
        item_Upplatelseform  = 0
        item_AntalRum        = 0
        item_BoArea          = 0
        item_Biarea          = 0
        item_Tomtarea        = 0
        item_byggAr          = 0
        item_PropertyFee     = 0
        
        for i in range(len(item_data_Categorynames)):
            if(item_data_Categorynames[i] == 'Bostadstyp'):
                item_Bostadstyp      = item_data_CategoryValues[i]
            elif(item_data_Categorynames[i] == 'Upplåtelseform'):
                item_Upplatelseform  = item_data_CategoryValues[i]
            elif(item_data_Categorynames[i] == 'Antal rum'):
                item_AntalRum        = item_data_CategoryValues[i]
            elif(item_data_Categorynames[i] == 'Boarea'):
                item_BoArea          = item_data_CategoryValues[i]
            elif(item_data_Categorynames[i] == 'Biarea'):
                item_Biarea          = item_data_CategoryValues[i]
            elif(item_data_Categorynames[i] == 'Tomtarea'):
                item_Tomtarea        = item_data_CategoryValues[i]
            elif(item_data_Categorynames[i] == 'Byggår'):
                item_byggAr          = item_data_CategoryValues[i]
            elif(item_data_Categorynames[i] == 'Tomträttsavgäld'):
                item_PropertyFee          = item_data_Categorynames[i]+" "+item_data_CategoryValues[i]
       
        #****************************************
        
        
        # Get the "Utgångspris" that is Starting price -IF THER EXISTS ONE
        
        try:
            item_Utgangpris_parent = soup.find('dl',{'class':'sold-property__price-stats'})#.text.strip()
            item_Utgangpris_child = item_Utgangpris_parent.find('dd').text.strip()
            item_Utgangpris_child = " ".join(re.split("\s+", item_Utgangpris_child, flags=re.UNICODE)).replace('m²','').replace('rum','')
        except AttributeError:
            item_Utgangpris_child = 0
        #**********************************************

             
        #---------------------------------------------------------
        # create a tuple with the all the data about the property 
        #---------------------------------------------------------
        data = (propertyID_hemnet,houseAddress_1,houseAddress_2, item_date_sold_at,item_finalprice,
                item_Bostadstyp, item_Upplatelseform, item_AntalRum, item_BoArea, item_Biarea, 
                item_Tomtarea, item_byggAr, item_PropertyFee, link)
        records_test4.append(data)
        
    try:    
        find_nextpage_url = soup.find('a', {'class' : 'next_page hcl-button hcl-button--primary hcl-button--full-width'})
        if find_nextpage_url.get('href'): # if there is a link
            url = 'https://www.hemnet.se/'+ find_nextpage_url.get('href')
            print(url)
        else:
            break
    except AttributeError: #if there is no next button link
        print("Done")
        break
    
columnNames = ['propertyID_hemnet','House_Address_1','House_address_2','date_sold_at','Slutpris','Bostadstyp', 
               'Upplatelseform', 'AntalRum', 'BoArea', 'Biarea', 
               'Tomtarea', 'byggAr', 'PropertyFee'    ,'link']
test4_table = pd.DataFrame(records_test4, columns= columnNames)

# eXPORT AS CSV
test4_table.to_csv('110properties_scrape_test.csv', sep=',', encoding='utf-8')

property number: 1 out of 110
Näckrosbrinken 18
property number: 2 out of 110
Ivarskärrsvägen 109 B (adress ej fastslaget)
property number: 3 out of 110
Sörgårdsvägen 65A
property number: 4 out of 110
Solrosvägen 86
property number: 5 out of 110
Solrosvägen 86
property number: 6 out of 110
Småbrukarvägen 17a
property number: 7 out of 110
Näckrosbrinken 10
property number: 8 out of 110
Solhemsbackarna 101B
property number: 9 out of 110
Åkerhielmsgatan 16B
property number: 10 out of 110
Svärdsliljevägen 24
property number: 11 out of 110
Sothönsgränd 2
property number: 12 out of 110
Adolf Rudbecks väg 16A
property number: 13 out of 110
Ryttmästarvägen 20
property number: 14 out of 110
Fadderortsgränd 4
property number: 15 out of 110
Stjärnfallsvägen 21
property number: 16 out of 110
Sothönsgränd 6
property number: 17 out of 110
Loviselundsvägen 108A
property number: 18 out of 110
Peterséns väg 24
property number: 19 out of 110
Blåklocksvägen 15
property number: 20 out of 110
Näckrosbrinke

In [555]:
item_date_sold_at = item_soup.find('time').text.strip()
item_date_sold_at

'6 december 2022'

In [548]:
columnNames = ['House_Address_1','House_address_2','date_sold_at','Slutpris','Bostadstyp', 
               'Upplatelseform', 'AntalRum', 'BoArea', 'Biarea', 
               'Tomtarea', 'byggAr', 'PropertyFee'    ,'link']
test4_table = pd.DataFrame(records_test4, columns= columnNames)

In [549]:
# export the test4_table as csv file
test4_table.to_csv('785properties_scrape_test.csv', sep='\t', encoding='utf-8')

In [571]:
link

'https://www.hemnet.se/salda/tomt-spanga-solhem-stockholms-kommun-avestagatan-39-664874'

In [575]:
link.split("-")[-1].strip()

'664874'

In [68]:

# loop through all items and keep track
records_test3 = []
# url = 'https://www.hemnet.se/salda/ovrigt-1rum-spanga-bromsten-stockholms-kommun-stridsbergavagen-2-4-9189425484463838645'
# url = 'https://www.hemnet.se/salda/villa-4rum-enskedefaltet-stockholms-kommun-finska-gatan-5-1017428'
# url = 'https://www.hemnet.se/salda/ovrigt-13rum-malarhojden-stockholms-kommun-slattgardsvagen-70a-1323883'
# url = 'https://www.hemnet.se/salda/villa-4rum-bromma-nockeby-stockholms-kommun-gronviksvagen-199-b-1038286'
# url that I got an error from luxparkens marina property because that only has 4 categories
url = 'https://www.hemnet.se/salda/villa-6rum-taserud-arvika-ostra-arvika-kommun-myrasvagen-1-1384867'
""" Set up the page"""

header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", 
"X-Amzn-Trace-Id": "Root=1-63c40070-5d3770130fb764901d13ba81"}

response = requests.get(url, headers=header)

soup = BeautifulSoup(response.content, 'html.parser')




In [23]:
results = soup.find_all('div', {'class': "column large"})

In [355]:
test = soup.find('span',{'class':'sold-property__price-value'})


In [328]:
test=test.text.strip()

In [329]:
" ".join(re.split("\s+", test, flags=re.UNICODE)).replace(" kr",'').replace(" ","")

'90000'

In [39]:
item_data = soup.find('dl',{'class':'sold-property__attributes'})#.text.strip()


In [71]:
soup.find('time').text

'\n      14 maj 2021\n    '

In [ ]:
item_date_sold_at = soup.find('time').text.strip()

### NEW way to manage all the different number of characteristics (categories) on each property page

In [40]:
item_data_dt = item_data.find_all('dt')
item_data_Categorynames = []
for category in item_data_dt:
    category = category.text
    item_data_Categorynames.append(category)
# item_data = " ".join(re.split("\s+", item_data, flags=re.UNICODE)).replace('m²','').replace('rum','').split()
item_data_Categorynames

['Bostadstyp',
 'Upplåtelseform',
 'Antal rum',
 'Boarea',
 'Biarea',
 'Tomtarea',
 'Byggår',
 'Driftskostnad']

In [41]:
item_data_dd = item_data.find_all('dd') # values
item_data_CategoryValues = []
for value in item_data_dd:
    value = value.text.strip()
    item_data_CategoryValues.append(value)
item_data_CategoryValues

['Villa',
 'Äganderätt',
 '4 rum',
 '81 m²',
 '73 m²',
 '623 m²',
 '1944',
 '41\xa0806\xa0kr/år']

In [42]:
item_Bostadstyp       = 0
item_Upplatelseform   = 0
item_AntalRum         = 0
item_BoArea           = 0
item_Biarea           = 0
item_Tomtarea         = 0
item_byggAr           = 0
#item_PropertyCost    = 0
item_AvgiftManad      = 0
item_Driftskostnad    = 0
item_Tomträttsavgäld  = 0
item_Balkong          = 0
item_Vaning           = 0
item_Uteplats         = 0

for i in range(len(item_data_Categorynames)):
    if(item_data_Categorynames[i] == 'Bostadstyp'):
        item_Bostadstyp      = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Upplåtelseform'):
        item_Upplatelseform  = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Antal rum'):
        item_AntalRum        = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Boarea'):
        item_BoArea          = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Biarea'):
        item_Biarea          = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Tomtarea'):
        item_Tomtarea        = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Byggår'):
        item_byggAr          = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Tomträttsavgäld'):
        item_Tomträttsavgäld = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Avgift/månad'):
        item_AvgiftManad     = item_data_CategoryValues[i]    
    elif(item_data_Categorynames[i] == 'Driftskostnad'):
        item_Driftskostnad   = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Balkong'):
        item_Balkong         = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Våning'):
        item_Vaning          = item_data_CategoryValues[i]
    elif(item_data_Categorynames[i] == 'Uteplats'):
        item_Uteplats        = item_data_CategoryValues[i]

        

        
        
    


In [43]:
print(f"item_Bostadstyp     : {item_Bostadstyp    }" )
print(f"item_Upplatelseform : {item_Upplatelseform}" )
print(f"item_AntalRum       : {item_AntalRum      }" )
print(f"item_BoArea         : {item_BoArea        }" )
print(f"item_Biarea         : {item_Biarea}" )
print(f"item_Tomtarea       : {item_Tomtarea      }" )
print(f"item_byggAr         : {item_byggAr        }" )
#print(f"item_PropertyCost   : {item_PropertyCost  }") 
print(f"item_AvgiftManad    : {item_AvgiftManad     }") 
print(f"item_Driftskostnad  : {item_Driftskostnad   }") 
print(f"item_Tomträttsavgäld: {item_Tomträttsavgäld }") 
print(f"item_Balkong        : {item_Balkong         }") 
print(f"item_Vaning         : {item_Vaning          }") 
print(f"item_Uteplats       : {item_Uteplats        }") 

item_Bostadstyp     : Villa
item_Upplatelseform : Äganderätt
item_AntalRum       : 4 rum
item_BoArea         : 81 m²
item_Biarea         : 73 m²
item_Tomtarea       : 623 m²
item_byggAr         : 1944
item_AvgiftManad    : 0
item_Driftskostnad  : 41 806 kr/år
item_Tomträttsavgäld: 0
item_Balkong        : 0
item_Vaning         : 0
item_Uteplats       : 0


### OLD way to manage all the different number of characteristics (categories) on each property page

In [453]:
# get all the other data with one call : 
    #  property type (Bostadstyp)   , Release form (Upplåtelseform), Number of rooms (Antal rum),
    #  size (Boarea)                , Biarea (Biarea)              , (Tomtarea), 
    #  Year of construction (Byggar), Fee/month (Avgift/månad), fee/year (Tomträttsavgäld)
# All at on fetch 
# item data: if we have 

if("Biarea" and "Tomtarea" in item_data):
    print("1")
    item_Bostadstyp      = item_data[1]
    item_Upplatelseform  = item_data[3]
    item_AntalRum        = item_data[5]
    item_BoArea          = item_data[7]
    item_Biarea          = item_data[9]
    item_Tomtarea        = item_data[11]
    item_byggAr          = item_data[13]
    item_PropertyFee     = ' '.join(item_data[14:])
elif("Biarea" in item_data):
    print("2")
    item_Bostadstyp      = item_data[1]
    item_Upplatelseform  = item_data[3]
    item_AntalRum        = item_data[5]
    item_BoArea          = item_data[7]
    item_Biarea          = item_data[9]
    item_Tomtarea        = 0
    item_byggAr          = item_data[11]
    item_PropertyFee     = ' '.join(item_data[12:])
elif ("Tomtarea" in item_data):
    print("3")
    item_Bostadstyp      = item_data[1]
    item_Upplatelseform  = item_data[3]
    item_AntalRum        = item_data[5]
    item_BoArea          = item_data[7]
    item_Biarea          = 0
    item_Tomtarea        = item_data[9]
    item_byggAr          = item_data[11]
    item_PropertyFee     = ' '.join(item_data[12:])
else: 
    print("4")
    item_Bostadstyp      = item_data[1]
    item_Upplatelseform  = item_data[3]
    item_AntalRum        = item_data[5]
    item_BoArea          = item_data[7]
    item_Biarea          = 0
    item_Tomtarea        = 0
    item_byggAr          = item_data[9]
    item_PropertyFee     = ' '.join(item_data[10:])
    


4


In [454]:
print(f"item_Bostadstyp     : {item_Bostadstyp    }" )
print(f"item_Upplatelseform : {item_Upplatelseform}" )
print(f"item_AntalRum       : {item_AntalRum      }" )
print(f"item_BoArea         : {item_BoArea        }" )
print(f"item_Biarea         : {item_Biarea        }" )
print(f"item_Tomtarea       : {item_Tomtarea      }" )
print(f"item_byggAr         : {item_byggAr        }" )
print(f"item_PropertyCost   : {item_PropertyCost  }") 

item_Bostadstyp     : Övrig
item_Upplatelseform : Bostadsrätt
item_AntalRum       : 1
item_BoArea         : 14
item_Biarea         : 0
item_Tomtarea       : 0
item_byggAr         : 1965
item_PropertyCost   : Avgift/månad 425 kr/mån


In [467]:
item_Utgangpris_parent = soup.find('dl',{'class':'sold-property__price-stats'})#.text.strip()
item_Utgangpris_child = item_Utgangpris_parent.find('dd').text.strip()
item_Utgangpris_child = " ".join(re.split("\s+", item_Utgangpris_child, flags=re.UNICODE)).replace('m²','').replace('rum','')
item_Utgangpris_child

'5 195 000 kr'

In [422]:
x = [["a","b"], ["c"]]

result = sum(x, [])
result

['a', 'b', 'c']

In [404]:
a = [1,2,3,4,5]

if((1 and 3) in a):
    print("yeah")
else:
    print("no")

yeah


In [16]:
import random
random.randint(1,10)

4

#  ########-**SETTING IT ALL TO GETHER**-#######

In [26]:
def search_bar(municipal_list):
    
    #Define empty list to collect count data
    #######################################
    listplace = []
    listStageNumber = []
    listStageName = []
    listCategoryMinmax = [] 
    listResult = []
    
    # Define all the search categories
    ##################################
    """Stage 2: Bostadstyp"""
    house_category = ['villa','radhus','bostadsratt','fritidshus','tomt','gard','other']
#     print(len(house_category))

    '''Stage 3: number of rooms'''
    rooms = [['min',1], [1.5,1.5], [2,2], [2.5,2.5], [3,3], [3.5,3.5], [4,5], [6,7], [8,15], [16,'max']]
#     print(len(rooms))

    '''Stage 4: Boarea/ size'''
    size = [['min',30], [31,40], [41,50], [51,60], [61,70], [71,80], [81,90], [91,100], [101,110], [111,120], [121,130], [131,140], [141,150], [151,160],[161,170], [171,180], [181,190], [191,200], [201,210], [211,220], [221,230], [231,240], [241,250], [251,'max']]
#     print(len(size))

    '''Stage 5: Avgift'''
    avgift = [["min",500], [501,1000], [1001,1500], [1501,2000], [2001,2500], [2501,3000], [3001,3500], [3501,4000], [4001,4500], [4501,5000],[5001,5500], [5501,6000],[6001,6500], [6501,7000], [7001,7500], [7501,8000], [8001,8500], [8501,9000], [9001,9500],[9501,10000], [10001,20000], [20001,'max']]
#     print(len(avgift))

    '''Stage 6: finalprice'''
    finalprice = [['min',1000000], [1000001,2000000], [2000001,3000000], [3000001,4000000], [4000001,5000000],[5000001,6000000], [6000001,7000000], [7000001,8000000], [8000001,9000000], [9000001,10000000], [10000001,11000000], [11000001,12000000], [12000001,13000000],[13000001,14000000], [14000001,15000000], [15000001,16000000], [16000001,'max']]
#     print(len(finalprice))
    
    # ----------------------
    # Set up the web-driver
    #----------------------
    driver = webdriver.Chrome(ChromeDriverManager().install())

    # Maximize the window - otherwise we will get problems with the accessing the search bar
    driver.maximize_window()

    base_url = 'https://www.hemnet.se/salda/bostader'
    while True:
        try:
            driver.get(base_url)
        except (TimeoutException,TimeoutError) as e:
            print("------TimeoutException,TimeoutError- BASE URL --- RETRY")
            continue
        break

    # sleep for 3 second then close the popup window
    time.sleep(3)
    
    # close the popup window
    try:
        driver.find_element(
            By.XPATH,"//button[@ class = 'hcl-button hcl-button--primary']").click()
    except Exception:
        pass 
    #time.sleep(5)
    
    for municipal_number in range(len(municipal_list)):
                
        area = municipal_list[municipal_number] 
        print(f"-----------------------------------------------> Municipality: {area} ")
        
        sleepNumber = random.randint(1,6)
        time.sleep(sleepNumber)
        
        while True:
            try:
                base_url = 'https://www.hemnet.se/salda/bostader'
                driver.get(base_url)
                time.sleep(4)
            except (TimeoutException, StaleElementReferenceException) as e:
                print("------TimeoutException,TimeoutError--- ENTER ---- RETRY")
                continue
            break


        # Search the given municipal (variable= area)
        content = driver.find_element(By.CLASS_NAME,"qa-autocomplete-location")
        content.clear()
        content.send_keys(area)
        time.sleep(2)
        #time.sleep(2) # GIVE IT TIME TO LOAD THE RESULT

        content.send_keys(Keys.ENTER)
        

        # press enter:
            # another way: simply press enter again because after the first enter the website takes you straight to the search button
        while True:
            try:
                content.send_keys(Keys.ENTER)
            except (TimeoutException, StaleElementReferenceException) as e:
                continue
            break
                
            

        # Original solutions - but sometimes the don't fully complete and just keep running
        #         driver.find_element(
        #             By.XPATH,"//button[@type = 'submit']"
        #         ).click()
#         print("pressing search button")
        #         driver.find_element(
        #             By.XPATH,"//button[@class='hcl-button hcl-button--primary hcl-button--full-width search-form__submit-button qa-search-form-submit-button']"
        #         ).click()

        
        # delay 3 seconds: 
        time.sleep(1)

        # Stage 1: Count the number of research results:
#         print("counting result")
        stage1_search_count = int(driver.find_element(
                                By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

        print(f'Stage 1 result count: {stage1_search_count}')
        
        #add to list
        listplace.append(area)
        listStageNumber.append(1)
        listStageName.append('Municipalities')
        listCategoryMinmax.append('0')
        listResult.append(stage1_search_count)

        # if search result is smaller than 2500 then scrape - now just print
#         print("if statement")
        if(stage1_search_count<=2500):
            print(f'Completet at stage 1. Search results in stage 1 = {stage1_search_count}')
            
            stage1_url = driver.current_url
            
            ################ Scrape !!!!
#             print("Scraping at stage 1")
            scrape(stage1_url,stage1_search_count,1,area)
            #####################
            
        # Stage 2: If stage 1 result count is greater than 2500 move to stage 2: rooms
        else:
#             print("stage 2")
          
            # Get the current url and set it as the base URL
            stage1_url = driver.current_url
            

            # Move to stage 2 - the categories
            for house_type in range(7):
                category = house_category[house_type]  

                # Set the URL to the category URL
                stage2_url = stage1_url + '&item_types%5B%5D='+category

                # Go to the website of the url:
                print()
                #time.sleep(3) 
                while True:                    
                    try: 
                        driver.set_page_load_timeout(3)
                        driver.get(stage2_url)
                        time.sleep(3)
                    except (TimeoutException,TimeoutError) as e:
                        print("------TimeoutException,TimeoutError---- RETRY")
                        continue
                    break

                # Count the number of research results of stage 2:
#                 print("counting results")
                stage2_search_count = int(driver.find_element(
                                           By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

                print(f'Stage 2, category {category} - result count: {stage2_search_count}')
                
                #add to list
                listplace.append(area)
                listStageNumber.append(2)
                listStageName.append('House Type')
                listCategoryMinmax.append(category)
                listResult.append(stage2_search_count)

                # if search result is smaller than 2500 then scrape - now just print
                print("if statement")
                if(stage2_search_count<=2500):
                    print(f'Completed at stage 2. Search results in stage 2 for category {category} = {stage2_search_count}')
                    
                    ################ Scrape !!!!
#                     print("Scraping at stage 2")
                    try:
                        scrape(stage2_url,stage2_search_count,2,area)
                    except ConnectionError:
                        scrape(stage2_url,stage2_search_count,2,area)
                    #####################
                    
                    
                # If not move to stage 3
                # Stage 3: If the given category in stage 2 has result count greater than 2500
                # move to stage 3: number of rooms
                else:
#                     print("stage 3")
                    # get current URL to add on it.
                    stage2_url = driver.current_url

                    for minmax in range(10):
                        room_minmax = rooms[minmax]

                        # Set the URL to the category URL
                        stage3_url = stage2_url + '&rooms_min='+ str(room_minmax[0]) + '&rooms_max='+ str(room_minmax[1])

                        # Go to the website of the url: 
#                         time.sleep(2)
                        while True:
                            try:
                                driver.get(stage3_url)
                                time.sleep(3)
                            except (TimeoutException,TimeoutError) as e:
                                print("------TimeoutException,TimeoutError---- RETRY")
                                continue
                            break
                            
                        
                        # Count the number of research results of stage 2:
#                         print("counting results")
                        stage3_search_count = int(driver.find_element(
                                                    By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

                        print(f'Stage 3, Room minmax set {room_minmax} - result count: {stage3_search_count}')
                        
                        #add to list
                        listplace.append(area)
                        listStageNumber.append(3)
                        listStageName.append('Number of Rooms')
                        listCategoryMinmax.append(str('MIN: '+str(room_minmax[0])+'-'+'MAX: '+str(room_minmax[1])))
                        listResult.append(stage3_search_count)

#                         print("if statement")
                        if(stage3_search_count<=2500):
                            print(f'Completed at stage 3. Search results in stage 3 for room minmax set {room_minmax} = {stage3_search_count}')
                            
                            ################ Scrape !!!!
#                             print("Scraping at stage 3")
                            scrape(stage3_url,stage3_search_count,3,area)
                            #####################
                            
                            

                        # Stage 4: If the given room minmax set in stage 3 has result count greater than 2500 
                        # move to stage 4 - size
                        else:
                            print("stage 4")

                            # get current URL to add on it.
                            stage3_url = driver.current_url

                            for minmax in range(24):
                                size_minmax = size[minmax]

                                # Set the URL to the category URL
                                stage4_url = stage3_url + '&living_area_min=' + str(size_minmax[0]) + '&living_area_max='+ str(size_minmax[1])

                                # Go to the website of the url:
                                
#                                 time.sleep(2) 
                                while True:
                                    try: 
                                        driver.set_page_load_timeout(3)
                                        driver.get(stage4_url)
                                        time.sleep(3)
                                    except (TimeoutException,TimeoutError) as e:
                                        print("------TimeoutException,TimeoutError---- RETRY")
                                        continue
                                    break

        
                                # Count the number of research results of stage 2:
#                                 print("counting results")
                                stage4_search_count = int(driver.find_element(
                                                            By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

                                print(f'Stage 4, Size minmax set {size_minmax} - result count: {stage4_search_count}')
                                
                                #add to list
                                listplace.append(area)
                                listStageNumber.append(4)
                                listStageName.append("")
                                listCategoryMinmax.append(str('MIN: '+str(size_minmax[0])+'-'+'MAX:'+str(size_minmax[1])))
                                listResult.append(stage4_search_count)

#                                 print("if statement")
                                if(stage4_search_count<=2500):
                                    print(f'Completed at stage 4. Search results in stage 4 for size minmax set {size_minmax} = {stage4_search_count}')
                                    
                                    ################ Scrape !!!!
                                    print("Scraping at stage 4")
                                    scrape(stage4_url,stage4_search_count,4,area)
                                    #####################

                                # Stage 5: If the given size minmax set in stage 4 has result count greater than 
                                # 2500 move to stage 5 -Avgift
                                else:
#                                     print("stage 5")
                                    # get current URL to add on it.
                                    stage4_url = driver.current_url

                                    for minmax in range(22):
                                        avgift_minmax = avgift[minmax]

                                        # Set the URL to the category URL
                                        stage5_url = stage4_url + '&fee_min=' + str(avgift_minmax[0]) + '&fee_max='+ str(avgift_minmax[1])

                                        # Go to the website of the url: 
#                                         time.sleep(2)
            
                                        while True:
                                            try: 
                                                driver.set_page_load_timeout(10)
                                                driver.get(stage5_url)
                                                time.sleep(3)
                                            except (TimeoutException,TimeoutError) as e:
                                                print("------TimeoutException,TimeoutError---- RETRY")
                                                continue
                                            break

                                        # Count the number of research results of stage 2:
#                                         print("counting results")
                                        stage5_search_count = int(driver.find_element(
                                                                    By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

                                        print(f'Stage 5, Avgift minmax set {avgift_minmax} - result count: {stage5_search_count}')
                                        
                                        #add to list
                                        listplace.append(area)
                                        listStageNumber.append(5)
                                        listStageName.append('Avgift')
                                        listCategoryMinmax.append(str('MIN: '+str(avgift_minmax[0])+'-'+'MAX: '+str(avgift_minmax[1])))
                                        listResult.append(stage5_search_count)

#                                         print("if statement")
                                        if(stage5_search_count<=2500):
                                            print(f'Completed at stage 4. Search results in stage 4 for Avgift minmax set {avgift_minmax} = {stage5_search_count}')
                                            
                                            ################ Scrape !!!!
#                                             print("Scraping at stage 5")
                                            scrape(stage5_url,stage5_search_count,5,area)
                                            #####################
                                            
                                        # Stage 6: If the given Avgift minmax set in stage 5 has result count greater than 
                                        # 2500 move to stage 5 - Slutpris / Finalprice
                                        else:
#                                             print("stage 6")
                                            # get current URL to add on it.
                                            stage5_url = driver.current_url

                                            for minmax in range(17):
                                                finalprice_minmax = finalprice[minmax]

                                                # Set the URL to the category URL
                                                stage6_url = stage5_url + '&selling_price_min=' + str(finalprice_minmax[0]) + '&selling_price_max='+ str(finalprice_minmax[1])
                                                
    
                                                # Go to the website of the url: 
#                                                 time.sleep(2)
                                                            
                                                while True:
                                                    try: 
                                                        driver.set_page_load_timeout(3)
                                                        driver.get(stage6_url)
                                                        time.sleep(3)
                                                    except (TimeoutException,TimeoutError) as e:
                                                        print("------TimeoutException,TimeoutError---- RETRY")
                                                        continue
                                                    break

                                                # Count the number of research results of stage 2:
#                                                 print("counting results")
                                                stage6_search_count = int(driver.find_element(
                                                                            By.XPATH,"//li[@data-result-type= 'sold']").text.replace("Slutpriser\n",'').replace(' ',''))

                                                print(f'Stage 6, Finalprice minmax set {finalprice_minmax} - result count: {stage6_search_count}')
                                                
                                                ################ Scrape !!!!
#                                                 print("Scraping at stage 6")
                                                scrape(stage6_url,stage6_search_count,6,area)                                                
                                                #####################
                                                
                                                #add to list
                                                listplace.append(area)
                                                listStageNumber.append(6)
                                                listStageName.append('Finalprice')
                                                listCategoryMinmax.append(str('MIN: '+str(finalprice_minmax[0])+'-'+'MAX: '+str(finalprice_minmax[1])))
                                                listResult.append(stage6_search_count)
    # Export the result as csv file
    tableToCSV = pd.DataFrame({'Place': listplace, 'Stage_number': listStageNumber,
                               'Stage_name': listStageName, 'Category/MinMax': listCategoryMinmax,
                               'Number_of_Results': listResult})
    # write csv
    tableToCSV.to_csv('countData_errorCheck_290Municipalities.csv',sep='\t', encoding='utf-8')
                                
    return tableToCSV 


In [4]:
def scrape(url,total_properties,stage_number,municipal_name):
    # loop through all items and keep track
    records_test4 = []
    #url = 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18031&item_types%5B%5D=tomt'
    
    property_count = 0
    
    while True:
        """ Set up the page"""
        
#         # ----------------------
#         # Set up the web-driver
#         #----------------------
#         driver = webdriver.Chrome(ChromeDriverManager().install())

#         # Maximize the window - otherwise we will get problems with the accessing the search bar
#         driver.maximize_window()

#         driver.get(url)

        # using web driver thorugh request package not Selenium;
        header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", 
        "X-Amzn-Trace-Id": "Root=1-63c40070-5d3770130fb764901d13ba81"}
        
        # session
        session = requests.Session()

        response = session.get(url, headers=header,verify=False)
#         time.sleep()
        soup = BeautifulSoup(response.content, 'html.parser')

        results4 = soup.find_all('li', {'class': "sold-results__normal-hit"})

        # *********************************************

        for item in results4:
            property_count += 1
#             print(f"------------------> property number: {property_count} out of {total_properties} ")

            # get the link of each item
            link = item.find('a', {'class': "sold-property-link js-sold-property-card-link"}).get('href')

            # Create property ID column - from the ID number in the end of each property URL
            propertyID_hemnet = link.split("-")[-1].strip()

            # get the house address of each item
            houseAddress_parent = item.find('div', {'class': 'sold-property-listing__location'})

            # get the house address 1
            houseAddress_1 = houseAddress_parent.find('h2').text.strip()
#             print(f'------------------> {houseAddress_1}')
            # get the house address 2
            houseAddress_2 = houseAddress_parent.find('div').text.strip().replace("\n","")[13:].strip()
                # import re
            houseAddress_2 = " ".join(re.split("\s+", houseAddress_2, flags=re.UNICODE)) # get rid of double white spaces

            #######################################
            #
            #  Go to property link and get more data
            #
            #######################################
            # keep getting Attribute error and i think its because of a stupid pop up window from hemnet asking
            # how we like the website - so i put everything in a while loop to make the hole code retry if this 
            # stupid window come on again instead of trying to find this window myself and figure out the selectors
            # on the close popup window because that will take loong time to wait for that window to show up
            while True:
                try:
                    item_url      = link
        #           time.sleep(1)
        #           item_response = requests.get(item_url, headers=header,verify=False)
                    item_response = session.get(item_url, headers=header,verify=False)
        #             time.sleep(5)
        #             time.sleep(3) # keep getting silly attributes errors - might need to let the browser to load the url properly
                    item_soup     = BeautifulSoup(item_response.content, 'html.parser')


                    # get the date the porperty was last sold
                    try:
                        item_date_sold_at = item_soup.find('time').text.strip()
                    except AttributeError:
                        print("Unable to get selling date of the following property:")
                        print(link)
                    #****************************************

                    # get the final price
                    try:
                        item_finalprice_raw = item_soup.find('span',{'class':'sold-property__price-value'}).text.strip()
                        item_finalprice = " ".join(re.split("\s+", item_finalprice_raw, flags=re.UNICODE)).replace(" kr",'').replace(" ","")
                    except AttributeError:
                        print("Unable to get final price of the following property:")
                        print(link)


                    #****************************************

                    # get all the other data with one call : 
                        #  property type (Bostadstyp)   , Release form (Upplåtelseform), Number of rooms (Antal rum),
                        #  size (Boarea)                , Biarea (Biarea)              ,  land area (Tomtarea), 
                        #  Year of construction (Byggar), Fee/month (Avgift/månad), fee/year (Tomträttsavgäld)
                    # All at on fetch - the key difference between properties is if there is "Biarea" and/or "Tomtarea"
                    # then the list is longer. And the last category is different e.g. Avgift/månad or Tomträttsavgäld etc. in per mount
                    # or in per year fees. 
                            # collect the data
                    item_data = item_soup.find('dl',{'class':'sold-property__attributes'})#.text.strip()

                            # set up the data to run the forloop of the categories
                    # Category names:
                    item_data_dt = item_data.find_all('dt')
                    item_data_Categorynames = []
                    for category in item_data_dt:
                        category = category.text
                        item_data_Categorynames.append(category)
                    item_data_Categorynames

                    # Category Values:
                    item_data_dd = item_data.find_all('dd') # values
                    item_data_CategoryValues = []
                    for value in item_data_dd:
                        value = value.text.strip()
                        item_data_CategoryValues.append(value)
                    item_data_CategoryValues

                    # Initiate a zero value (0) to all the categories before setting the actual values in.
                        # Because if a property does not e.g. have "byggar" then at least it will be 0 instead of plank
                    item_Bostadstyp       = 0
                    item_Upplatelseform   = 0
                    item_AntalRum         = 0
                    item_BoArea           = 0
                    item_Biarea           = 0
                    item_Tomtarea         = 0
                    item_byggAr           = 0
                    #item_PropertyCost    = 0
                    item_AvgiftManad      = 0
                    item_Driftskostnad    = 0
                    item_Tomträttsavgäld  = 0
                    item_Balkong          = 0
                    item_Vaning           = 0
                    item_Uteplats         = 0

                    for i in range(len(item_data_Categorynames)):
                        if(item_data_Categorynames[i] == 'Bostadstyp'):
                            item_Bostadstyp      = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Upplåtelseform'):
                            item_Upplatelseform  = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Antal rum'):
                            item_AntalRum        = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Boarea'):
                            item_BoArea          = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Biarea'):
                            item_Biarea          = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Tomtarea'):
                            item_Tomtarea        = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Byggår'):
                            item_byggAr          = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Tomträttsavgäld'):
                            item_Tomträttsavgäld = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Avgift/månad'):
                            item_AvgiftManad     = item_data_CategoryValues[i]    
                        elif(item_data_Categorynames[i] == 'Driftskostnad'):
                            item_Driftskostnad   = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Balkong'):
                            item_Balkong         = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Våning'):
                            item_Vaning          = item_data_CategoryValues[i]
                        elif(item_data_Categorynames[i] == 'Uteplats'):
                            item_Uteplats        = item_data_CategoryValues[i]

                    #****************************************


                    # Get the "Utgångspris" that is Starting price -IF THER EXISTS ONE

                    try:
                        item_Utgangpris_parent = item_soup.find('dl',{'class':'sold-property__price-stats'})#.text.strip()
                        item_Utgangpris_child = item_Utgangpris_parent.find('dd').text.strip()
                        item_Utgangpris_child = " ".join(re.split("\s+", item_Utgangpris_child, flags=re.UNICODE)).replace('m²','').replace('rum','')
#                         print(item_Utgangpris_child)
                    except AttributeError:
                        item_Utgangpris_child = 0
                    #**********************************************


                    #---------------------------------------------------------
                    # create a tuple with the all the data about the property 
                    #---------------------------------------------------------
                    data = (propertyID_hemnet   , houseAddress_1       , houseAddress_2       , item_date_sold_at,
                            item_finalprice     , item_Utgangpris_child,item_Bostadstyp       , item_Upplatelseform  ,    
                            item_AntalRum       , item_BoArea          , item_Biarea          , item_Tomtarea   , 
                            item_byggAr         , item_Tomträttsavgäld , item_AvgiftManad     , item_Driftskostnad , 
                            item_Balkong        , item_Vaning          , item_Uteplats        , link)
                    records_test4.append(data)
                    
                except AttributeError:
                    print('RETRY ----------------------------')
                    continue
                break # brake from the while loop
            # ____________________________________________
            # Go into the property link and get more data
            #_____________________________________________
                    
        
        
        # Go to next page if it exists:
        try:    
            find_nextpage_url = soup.find('a', {'class' : 'next_page hcl-button hcl-button--primary hcl-button--full-width'})
            if find_nextpage_url.get('href'): # if there is a link
                url = 'https://www.hemnet.se/'+ find_nextpage_url.get('href')
#                 print(f'------------------> {url}')
#                 time.sleep(3)
                
            else:
                break
        except AttributeError: #if there is no next button link
            print("------------------>Done")
            break

    columnNames = [ 'propertyID_hemnet', 'houseAddress_1 '      , 'houseAddress_2'     , 'item_date_sold_at',
                    'item_finalprice'  , 'item_Utgangpris_child', 'item_Bostadstyp'    , 'item_Upplatelseform', 
                    'item_AntalRum'    , 'item_BoArea'          , 'item_Biarea'        , 'item_Tomtarea',
                    'item_byggAr'      , 'item_Tomträttsavgäld' , 'item_AvgiftManad'   , 'item_Driftskostnad ',
                    'item_Balkong'     , 'item_Vaning'          , 'item_Uteplats'      , 'link']
    test4_table = pd.DataFrame(records_test4, columns= columnNames)

    # EXPORT AS CSV
    filename_base = "Municipal_{}_Stage_{}_properties_{}.csv"
    input_filename = filename_base.format(municipal_name, stage_number, total_properties)
    test4_table.to_csv(input_filename, sep=',', encoding='utf-8')

In [5]:
def main():

    # ---------------------------------
    # Get list of Swedish Municipalities
    #-----------------------------------
    path = "Sweden_municipalities_data/Sweden_municipalities_main.csv"

    # read the data as a list - easier to loop through it than a pandas frame
    municipalities = pd.read_csv(path,encoding="UTF-8").Kommun.tolist()
    
    # ----------------------
    # Run the program
    #----------------------
    search_bar(municipalities)
    
    # finished
    print("Process Finished: All 290 Manicipalities of Sweden have been scraped")


In [28]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

main()

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:05<00:00, 1.19MB/s]


-----------------------------------------------> Municipality: Lessebo kommun 
Stage 1 result count: 564
Completet at stage 1. Search results in stage 1 = 564
------------------>Done
-----------------------------------------------> Municipality: Lidingö kommun 
Stage 1 result count: 6193

Stage 2, category villa - result count: 910
if statement
Completed at stage 2. Search results in stage 2 for category villa = 910
------------------>Done

Stage 2, category radhus - result count: 596
if statement
Completed at stage 2. Search results in stage 2 for category radhus = 596


ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

### ATTENTION: Dreifi verkefnum á tölvunar með eftirfarandi hætti:

* **Thinkpad  :** From Ale       Kommun   to  Munkedals   kommun
* **Macbook   :** From Munkfors  kommun   to  Töreboda    kommun
* **Acer      :** From Uddevalla kommun   to  Övertornea  kommun



HIN TÖLVAN KEYRIR FRÁ **SIGTUNA KOMMUN** TIL **ÖRVERTORNEA KOMMUN** - 108 stykki

In [ ]:
error 6: 

In [ ]:
TimeoutException                          Traceback (most recent call last)
~\AppData\Local\Temp\ipykernel_7860\451043146.py in <module>
----> 1 main()

~\AppData\Local\Temp\ipykernel_7860\3568317436.py in main()
     12     # Run the program
     13     #----------------------
---> 14     search_bar(municipalities)
     15 
     16     # finished

~\AppData\Local\Temp\ipykernel_7860\2527856442.py in search_bar(municipal_list)
    221                                 # Go to the website of the url:
    222 #                                 time.sleep(2)
--> 223                                 driver.get(stage4_url)
    224                                 time.sleep(3)
    225 

~\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in get(self, url)
    331         Loads a web page in the current browser session.
    332         """
--> 333         self.execute(Command.GET, {'url': url})
    334 
    335     @property

~\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in execute(self, driver_command, params)
    319         response = self.command_executor.execute(driver_command, params)
    320         if response:
--> 321             self.error_handler.check_response(response)
    322             response['value'] = self._unwrap_value(
    323                 response.get('value', None))

~\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py in check_response(self, response)
    240                 alert_text = value['alert'].get('text')
    241             raise exception_class(message, screen, stacktrace, alert_text)
--> 242         raise exception_class(message, screen, stacktrace)
    243 
    244     def _value_or_default(self, obj, key, default):

TimeoutException: Message: timeout: Timed out receiving message from renderer: -1086.019
  (Session info: chrome=109.0.5414.75)

error 5: same as error 4 but different property:

In [ ]:
# error 5: same as error 4 but different property:
        
# ------------------> property number: 949 out of 1569 
# ------------------> Gränsvägen 8 & 10
# C:\Users\ABC\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.hemnet.se'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
#   warnings.warn(
# Unable to get selling date of the following property:
# https://www.hemnet.se/salda/villa-5rum-erikslund-fagernas-bodens-kommun-gransvagen-8-10-660942
# Unable to get final price of the following property:
# https://www.hemnet.se/salda/villa-5rum-erikslund-fagernas-bodens-kommun-gransvagen-8-10-660942
# ---------------------------------------------------------------------------
# AttributeError                            Traceback (most recent call last)
# ~\AppData\Local\Temp\ipykernel_7860\451043146.py in <module>
# ----> 1 main()

# ~\AppData\Local\Temp\ipykernel_7860\1829352033.py in main()
#      12     # Run the program
#      13     #----------------------
# ---> 14     search_bar(municipalities),propertyID_hemnet,houseAddress_1 ,houseAddress_2,item_date_sold_at,item_finalprice,item_Utgangpris_child,item_Bostadstyp,item_Upplatelseform,item_AntalRum,item_BoArea,item_Biarea,item_Tomtarea,item_byggAr,item_Tomträttsavgäld,item_AvgiftManad,item_Driftskostnad ,item_Balkong,item_Vaning,item_Uteplats,link


# ~\AppData\Local\Temp\ipykernel_7860\2956840959.py in search_bar(municipal_list)
#     157                     ################ Scrape !!!!
#     158                     print("Scraping at stage 2")
# --> 159                     scrape(stage2_url,stage2_search_count,2,area)
#     160                     #####################
#     161 

# ~\AppData\Local\Temp\ipykernel_7860\2778150527.py in scrape(url, total_properties, stage_number, municipal_name)
#      98                     # set up the data to run the forloop of the categories
#      99             # Category names:
# --> 100             item_data_dt = item_data.find_all('dt')
#     101             item_data_Categorynames = []
#     102             for category in item_data_dt:

# AttributeError: 'NoneType' object has no attribute 'find_all'

error 4: Think that the reason is because of a pop-up window with some questions.....................


In [ ]:
# Unable to get selling date of the following property:
# https://www.hemnet.se/salda/lagenhet-1rum-bjuv-bjuvs-kommun-prastgatan-10-d-303025
# Unable to get final price of the following property:
# https://www.hemnet.se/salda/lagenhet-1rum-bjuv-bjuvs-kommun-prastgatan-10-d-303025


# C:\Users\ABC\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.hemnet.se'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
#   warnings.warn(
# ---------------------------------------------------------------------------
# AttributeError                            Traceback (most recent call last)
# ~\AppData\Local\Temp\ipykernel_12676\451043146.py in <module>
# ----> 1 main()

# ~\AppData\Local\Temp\ipykernel_12676\1829352033.py in main()
#      12     # Run the program
#      13     #----------------------
# ---> 14     search_bar(municipalities)

# ~\AppData\Local\Temp\ipykernel_12676\2956840959.py in search_bar(municipal_list)
#     112             ################ Scrape !!!!
#     113             print("Scraping at stage 1")
# --> 114             scrape(stage1_url,stage1_search_count,1,area)
#     115             #####################
#     116 

# ~\AppData\Local\Temp\ipykernel_12676\1045677510.py in scrape(url, total_properties, stage_number, municipal_name)
#      93                     # set up the data to run the forloop of the categories
#      94             # Category names:
# ---> 95             item_data_dt = item_data.find_all('dt')
#      96             item_data_Categorynames = []
#      97             for category in item_data_dt:

# AttributeError: 'NoneType' object has no attribute 'find_all'

In [62]:
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", 
        "X-Amzn-Trace-Id": "Root=1-63c40070-5d3770130fb764901d13ba81"}

item_url      = 'https://www.hemnet.se/salda/lagenhet-1rum-bjuv-bjuvs-kommun-prastgatan-10-d-303025'
#             time.sleep(1)
item_response = requests.get(item_url, headers=header,verify=False)
#             time.sleep(3) # keep getting silly attributes errors - might need to let the browser to load the url properly
item_soup     = BeautifulSoup(item_response.content, 'html.parser')


# get the date the porperty was last sold
try:
    item_date_sold_at = item_soup.find('time').text.strip()
except AttributeError:
    print("Unable to get selling date of the following property:")
    print(link)
#****************************************

# get the final price
try:
    item_finalprice_raw = item_soup.find('span',{'class':'sold-property__price-value'}).text.strip()
    item_finalprice = " ".join(re.split("\s+", item_finalprice_raw, flags=re.UNICODE)).replace(" kr",'').replace(" ","")
except AttributeError:
    print("Unable to get final price of the following property:")
    print(link)




C:\Users\ABC\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.hemnet.se'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [64]:
#****************************************

# get all the other data with one call : 
    #  property type (Bostadstyp)   , Release form (Upplåtelseform), Number of rooms (Antal rum),
    #  size (Boarea)                , Biarea (Biarea)              ,  land area (Tomtarea), 
    #  Year of construction (Byggar), Fee/month (Avgift/månad), fee/year (Tomträttsavgäld)
# All at on fetch - the key difference between properties is if there is "Biarea" and/or "Tomtarea"
# then the list is longer. And the last category is different e.g. Avgift/månad or Tomträttsavgäld etc. in per mount
# or in per year fees. 
        # collect the data
item_data = item_soup.find('dl',{'class':'sold-property__attributes'})#.text.strip()
item_data

<dl class="sold-property__attributes">
<dt class="sold-property__attribute">Bostadstyp</dt>
<dd class="sold-property__attribute-value">Lägenhet</dd>
<dt class="sold-property__attribute">Upplåtelseform</dt>
<dd class="sold-property__attribute-value">Bostadsrätt</dd>
<dt class="sold-property__attribute">Antal rum</dt>
<dd class="sold-property__attribute-value">1 rum</dd>
<dt class="sold-property__attribute">Boarea</dt>
<dd class="sold-property__attribute-value">42 m²</dd>
<dt class="sold-property__attribute">Byggår</dt>
<dd class="sold-property__attribute-value">
          1990
        </dd>
<dt class="sold-property__attribute">Avgift/månad</dt>
<dd class="sold-property__attribute-value">3 227 kr/mån</dd>
<dt class="sold-property__attribute">Förening</dt>
<dd class="sold-property__attribute-value">
          Brf Prästgården
        </dd>
</dl>

In [65]:

        # set up the data to run the forloop of the categories
# Category names:
item_data_dt = item_data.find_all('dt')
item_data_Categorynames = []
for category in item_data_dt:
    category = category.text
    item_data_Categorynames.append(category)
item_data_Categorynames


['Bostadstyp',
 'Upplåtelseform',
 'Antal rum',
 'Boarea',
 'Byggår',
 'Avgift/månad',
 'Förening']

In [66]:

# Category Values:
item_data_dd = item_data.find_all('dd') # values
item_data_CategoryValues = []
for value in item_data_dd:
    value = value.text.strip()
    item_data_CategoryValues.append(value)
item_data_CategoryValues


['Lägenhet',
 'Bostadsrätt',
 '1 rum',
 '42 m²',
 '1990',
 '3\xa0227\xa0kr/mån',
 'Brf Prästgården']

error 3: 

In [46]:
driver = webdriver.Chrome(ChromeDriverManager().install())

# Maximize the window - otherwise we will get problems with the accessing the search bar
driver.maximize_window()

base_url = 'https://www.hemnet.se/salda/bostader'
driver.get(base_url)

# sleep for 3 second then close the popup window
time.sleep(3)

# close the popup window
try:
    driver.find_element(
        By.XPATH,"//button[@ class = 'hcl-button hcl-button--primary']").click()
except Exception:
    pass 
#time.sleep(5)




In [50]:

area = 'Bergs kommun' 
print(f"-----------------------------------------------> Municipality: {area} ")

base_url = 'https://www.hemnet.se/salda/bostader'
driver.get(base_url)
#time.sleep(4)


# Search the given municipal (variable= area)
content = driver.find_element(By.CLASS_NAME,"qa-autocomplete-location")
content.clear()
content.send_keys(area)
time.sleep(2)

#time.sleep(2) # GIVE IT TIME TO LOAD THE RESULT

content.send_keys(Keys.ENTER)

# press enter:
    # another way: simply press enter again because after the first enter the website takes you straight to the search button
content.send_keys(Keys.ENTER)

# Original solutions - but sometimes the don't fully complete and just keep running
#         driver.find_element(
#             By.XPATH,"//button[@type = 'submit']"
#         ).click()
print("pressing search button")
#         driver.find_element(
#             By.XPATH,"//button[@class='hcl-button hcl-button--primary hcl-button--full-width search-form__submit-button qa-search-form-submit-button']"
#         ).click()


-----------------------------------------------> Municipality: Bergs kommun 
pressing search button


error 2: - set a time.sleep for 3 sec - might be that the driver hasn't finsihed loading the site properly

In [ ]:
# ------------------> property number: 729 out of 2273 
# ------------------> Etydvägen 10
#  https://www.hemnet.se/salda/villa-7rum-ingesund-arvika-kommun-etydvagen-10-1046138

# ---------------------------------------------------------------------------
# AttributeError                            Traceback (most recent call last)
# ~\AppData\Local\Temp\ipykernel_18128\451043146.py in <module>
# ----> 1 main()

# ~\AppData\Local\Temp\ipykernel_18128\2995067238.py in main()
#      12     # Run the program
#      13     #----------------------
# ---> 14     search_bar(municipalities)

# ~\AppData\Local\Temp\ipykernel_18128\2393673366.py in search_bar(municipal_list)
#     110             ################ Scrape !!!!
#     111             print("Scraping at stage 1")
# --> 112             scrape(stage1_url,stage1_search_count,1,area)
#     113             #####################
#     114 

# ~\AppData\Local\Temp\ipykernel_18128\1367682072.py in scrape(url, total_properties, stage_number, municipal_name)
#      70 
#      71             # get the final price
# ---> 72             item_finalprice_raw = item_soup.find('span',{'class':'sold-property__price-value'}).text.strip()
#      73             item_finalprice = " ".join(re.split("\s+", item_finalprice_raw, flags=re.UNICODE)).replace(" kr",'').replace(" ","")
#      74             #****************************************

# AttributeError: 'NoneType' object has no attribute 'text'

error 1:

In [77]:
# AttributeError :
# ------------------> property number: 315 out of 2273 
# ------------------> Myråsvägen 1


# ---------------------------------------------------------------------------
# AttributeError                            Traceback (most recent call last)
# ~\AppData\Local\Temp\ipykernel_18128\451043146.py in <module>
# ----> 1 main()

# ~\AppData\Local\Temp\ipykernel_18128\2995067238.py in main()
#      12     # Run the program
#      13     #----------------------
# ---> 14     search_bar(municipalities)

# ~\AppData\Local\Temp\ipykernel_18128\2393673366.py in search_bar(municipal_list)
#     110             ################ Scrape !!!!
#     111             print("Scraping at stage 1")
# --> 112             scrape(stage1_url,stage1_search_count,1,area)
#     113             #####################
#     114 

# ~\AppData\Local\Temp\ipykernel_18128\3369669090.py in scrape(url, total_properties, stage_number, municipal_name)
#      62 
#      63             # get the date the porperty was last sold
# ---> 64             item_date_sold_at = item_soup.find('time').text.strip()
#      65             #****************************************
#      66 

# AttributeError: 'NoneType' object has no attribute 'text'




Error testing:

In [82]:
item_url      = 'https://www.hemnet.se/salda/villa-7rum-ingesund-arvika-kommun-etydvagen-10-1046138'
#             time.sleep(1)
item_response = requests.get(item_url, headers=header,verify=False)
item_soup     = BeautifulSoup(item_response.content, 'html.parser')


C:\Users\ABC\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.hemnet.se'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [85]:
item_finalprice_raw = item_soup.find('span',{'class':'sold-property__price-value'}).text.strip()
item_finalprice = " ".join(re.split("\s+", item_finalprice_raw, flags=re.UNICODE)).replace(" kr",'').replace(" ","")
item_finalprice

'2100000'

In [83]:

# get the date the porperty was last sold
item_date_sold_at = item_soup.find('time').text.strip()
item_date_sold_at

'17 juli 2019'